In [1]:
import os
import subprocess
import tempfile
import pickle
import pandas as pd
import warnings
import matplotlib
#matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import sonnet as snt
import tensorflow as tf
import tarfile

from six.moves import cPickle
from six.moves import urllib
from six.moves import xrange
from simrna_drifts import *
from tensorflow.python.client import device_lib
import os    
import rna_enc_pdb
from mpl_toolkits.mplot3d import Axes3D

/software/tensorflow/1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/software/tensorflow/1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/software/tensorflow/1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/software/tensorflow/1.14/lib/python3.6/site-packages/tensorflow/python/f

8


In [2]:
#!ls -ltr /gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/vqvae100_3d_8/ckpt

In [3]:
#!cp /gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/vqvae100_3d_8/ckpt/plots/*loss* .

In [4]:
from __future__ import print_function

enc_seq_pdb   = rna_enc_pdb.RNAEncoder(L=100, args="seq")

ids_puzzles = ['3mei', '3p59', '3owz', '3v7e', '4p9r', '4gxy', '4r4v', '4l81', '5kpy', 
           '4lck', '5lys', '5lyu', '5lyv', '5iem', '4qlm', '4xw7', '5ddo', '5di4', 
           '5k7c', '5tpy', '5t5a', '5y85', '5y87', '5nwq', '5nz6', '6jq5', '6jq6',
           '6e8u', '6ol3', '6p2h', '6pmo', '6pom', '6ufm']



os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'#8,9'

##### SET NEW MAXIMAL SEQ LENGTH !!! #####
PARAMS               = {}
PARAMS['batch_size'] = 1
PARAMS['RNA_len']    = max_len = 100
PARAMS['max_depth']  = 64 # 8*8 atom level
PARAMS['N_GPUS'] = 1

# Set hyper-parameters.
batch_size = PARAMS['batch_size']
image_size = PARAMS['RNA_len'] 

# 100k steps should take < 30 minutes on a modern (>= 2017) GPU.
num_training_updates = 50 

# 2901 ckpt
num_hiddens = 64 
num_residual_hiddens = 12#32
num_residual_layers = 12#12

num_hiddens = 16
num_residual_hiddens = 4
num_residual_layers = 4


#01042020 
embedding_dim = 8   #PARAMS['max_depth']
#embedding_dim = 12   #PARAMS['max_depth']
# The higher this value, the higher the capacity in the information bottleneck.
num_embeddings = 3
sub_str = '_L100_lre5_emb'+str(num_embeddings)+'_dim'+str(embedding_dim)

head_ckpt_dir = '/gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/01022022/ckpt/'
head_sopran_dir = '/gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/01022022/'
head_ckpt_dir = '/gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/01022022/ckpt/'
head_ckpt_dir = '/sopran/shared/embeddings/vqvae_3d_100_0707/'

head_sopran_dir = '/gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/vqvae100_3d_8/'

with open( '/gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/L64x64_substructure_23022022.pkl', 'rb') as f:
    traindata_df, valdata_df, testdata_df = pickle.load( f)
print(len(traindata_df))



atom_list = ["P","C4'","C2-GA_Pur","C6","N9","C2-CU_Pyr","C4","N1"]
atid = dict(zip(atom_list, np.arange(len(atom_list)))) # atom to index dict
NUM_ATOMS = len(atom_list)
print(NUM_ATOMS)

def map_idx(i):
    """
    developed in /alto/chris/projects/main/rna_3d/reconstruct_3d_from_distances/20200728_reverse_symmetrization.ipynb
    """
    within_index  = i % NUM_ATOMS
    between_index = i//NUM_ATOMS
    return within_index*NUM_ATOMS + between_index

symmetrization_indices = map_idx(np.arange(NUM_ATOMS**2))

def symmetrize_LxLx64(D):
    """
    developed in /alto/chris/projects/main/rna_3d/reconstruct_3d_from_distances/20200728_reverse_symmetrization.ipynb
    input:  D [L,L,64]
    output: D [L,L,64]
    
    If this function is applied to an unsymmetrized tensor, it will be symmetrized.
    If this function is applied to a symmetrized tensor, it will be un-symmetrized!
    
    """    
    upper = np.transpose(np.triu(np.transpose(D,[2,0,1]), 0),[1,2,0])  # 0 includes diagonal
    lower = np.transpose(np.tril(np.transpose(D,[2,0,1]), -1),[1,2,0]) # 0 includes diagonal
    return upper + lower[:,:,symmetrization_indices]


def Lx3_coords_to_LxLx64_tensor(array_a, *args):

    """
    !!! THIS FUNCTION WAS FORMERLY CALLED get_3D_tensor !!!
    
    originally developed in:
    /export/chrisblum/projects/main/rna_3d/data_loaders_and_conversion/20200713_develop_data_augmentation_data_loader.ipynb
    and modified in:
    /alto/chris/projects/main/rna_3d/reconstruct_3d_from_distances/20200728_LL64_to_L3_Julius_reconstructions_faster_mirrorfix_symmetrizationfix.ipynb    

    input(s):
        either 
            coords [Ra*8, 3] and optionally [Rb*8, 3]
        or
            mask [Ra*8] and optionally [Rb*8]
        
    output:
        either
            D [Ra, Ra, 64]
        or
            D [Ra, Rb, 64]
    """
    

    if len(args)==1:
        array_b = args[0]
    else:
        array_b = array_a
    
    array_a_shape = array_a.shape
    array_b_shape = array_b.shape
    if array_a_shape[0]%NUM_ATOMS!=0:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_a_shape)
    if array_b_shape[0]%NUM_ATOMS!=0:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_b_shape)        
    
    if len(array_a_shape)!=len(array_b_shape):
        raise Exception('Inputs have different dimensions:', array_a_shape, array_b_shape)  
    
    ### assume that input was "coords"
    if len(array_a_shape)==2:
        Ra = len(array_a)//NUM_ATOMS
        Rb = len(array_b)//NUM_ATOMS
        A = array_a.reshape([Ra, 1, NUM_ATOMS, 1, 3]) # [R*8, 3] -> [R, 1, 8, 1, 3]
        B = array_b.reshape([1, Rb, 1, NUM_ATOMS, 3]) # [R*8, 3] -> [1, R, 1, 8, 3]
        
        D = np.sqrt(np.sum((A-B)**2, axis=4, keepdims=True)) # [R, R, 8, 8, 1]
        D = np.reshape(D, [Ra,Rb,NUM_ATOMS**2]) # [R, R, 8, 8, 1] -> [R, R, 64]
        
        
    ### assume that input was "mask"
    elif len(array_a_shape)==1:
        Ra = len(array_a)//NUM_ATOMS
        Rb = len(array_b)//NUM_ATOMS
        A = array_a.reshape([Ra, 1, NUM_ATOMS, 1]) # [R*8] -> [R, 1, 8, 1]
        B = array_b.reshape([1, Rb, 1, NUM_ATOMS]) # [R*8] -> [1, R, 1, 8] 
        D = A*B
        D = np.reshape(D, [Ra,Rb,NUM_ATOMS**2])
        
        
    else:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_shape)
    
    
    ### symmetrize
    if Ra==Rb:
        return symmetrize_LxLx64(D)
    return D

def LxLx64_tensor_to_L8xL8_matrix(tensor):

    """
    Convert flat (L x L x 64) matrix to big (L*8 x L*8) matrix
    input:  tensor [L,L,64]
    output: matrix [L*8, L*8]
    """

    L8 = tensor.shape[0] * NUM_ATOMS

    matrix = np.zeros([L8,L8])
    idx = np.arange(0, L8, NUM_ATOMS)
    ctr = 0
    for i in range(0,NUM_ATOMS):
        for j in range(0,NUM_ATOMS):
            idx_i = idx + i
            idx_j = idx + j
            putmask1 = np.zeros([L8,L8])
            putmask1[idx_i]=1
            putmask2 = np.zeros([L8,L8])
            putmask2[:,idx_j]=1
            
            matrix[np.logical_and(putmask1, putmask2)] = tensor[:,:,ctr].reshape(-1)
            ctr += 1
    return matrix


def matrix_to_coords(D):
    """
    developed in 20201002_distance_matrix_to_coordinates_multidimensional_scaling.ipynb
    
    input:  D (distance matrix)
    output: x (reconstructed coordinates)
    """
    
    D_sq = D**2
    M = 0.5 * (D_sq[[0],:] + D_sq[:,[0]] - D_sq)
    q,v = np.linalg.eigh(M) # numpy.linalg.eig often produces complex number with very small imaginary part.
    x = np.sqrt(np.reshape(q[-3:],[1,3])) * v[:,-3:]
    x = x[:,::-1] # numpy.linalg.eigh has largest eigenvalues last, so we need to flip the vector here...
    
    return x


def L8xL8_matrix_to_Lx3_coords(matrix, mask):

    matrix_reduced = matrix[mask==1,:][:,mask==1]
    coords_reduced = matrix_to_coords(matrix_reduced)
    
    L8 = matrix.shape[0]
    coords = np.zeros([L8,3])    
    coords[mask==1] = coords_reduced
    return coords


def superimposer(x,y, mask=None):
    
    if mask is not None:
        x = x[mask==1]
        y = y[mask==1]
        
    xn = x - np.mean(x,axis=0)
    yn = y - np.mean(y,axis=0)    
    
    C = np.matmul(xn.T,yn)

    U, s, V = np.linalg.svd(C, full_matrices=1, compute_uv=1)
    R = np.matmul(U,V)
    
    yn_proj = np.matmul(yn, R.T)
    
    diff = xn - yn_proj
    rms = np.sqrt(sum(sum(diff * diff)) / x.shape[0])
    
    
    return rms, xn, yn_proj


def get_mask_for_nucleotide(nuc):
    
    """
    input: nucleotide (string; either 'A', 'C', 'G', 'U')
    output: mask (numpy array of shape [8])
    
    Atoms are in the following order:
    0   P
    1   C4'a
    2   C2-GA_Pur
    3   C6
    4   N9
    5   C2-CU_Pyr
    6   C4
    7   N1
    """
    
    if nuc not in list('ACGU'):
        raise Exception('Nucleotide must be either A, C, G or U but is:', nuc)
        
    mask = np.zeros([8])
    mask[:2] = 1
    if nuc in list('GA'):
        mask[2:5]=1
        return mask
    elif nuc in list('CU'):
        mask[5:]=1
        return mask
    


# In[38]:


def get_mask_for_sequence(seq):
    return np.concatenate([get_mask_for_nucleotide(nuc) for nuc in seq], axis=0)




def get_3D_tensor(array_a, *args):
    
    """
    originally developed in:
    /export/chrisblum/projects/main/rna_3d/data_loaders_and_conversion/20200713_develop_data_augmentation_data_loader.ipynb
    and modified in:
    /alto/chris/projects/main/rna_3d/reconstruct_3d_from_distances/20200728_LL64_to_L3_Julius_reconstructions_faster_mirrorfix_symmetrizationfix.ipynb
    """
    
    """
    input(s):
        either 
            coords [Ra*8, 3] and optionally [Rb*8, 3]
        or
            mask [Ra*8] and optionally [Rb*8]
        
    output:
        either
            D [Ra, Ra, 64]
        or
            D [Ra, Rb, 64]
    """
    

    if len(args)==1:
        array_b = args[0]
    else:
        array_b = array_a
    
    array_a_shape = array_a.shape
    array_b_shape = array_b.shape
    if array_a_shape[0]%NUM_ATOMS!=0:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_a_shape)
    if array_b_shape[0]%NUM_ATOMS!=0:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_b_shape)        
    
    if len(array_a_shape)!=len(array_b_shape):
        raise Exception('Inputs have different dimensions:', array_a_shape, array_b_shape)  
    
    ### assume that input was "coords"
    if len(array_a_shape)==2:
        Ra = len(array_a)//NUM_ATOMS
        Rb = len(array_b)//NUM_ATOMS
        A = array_a.reshape([Ra, 1, NUM_ATOMS, 1, 3]) # [R*8, 3] -> [R, 1, 8, 1, 3]
        B = array_b.reshape([1, Rb, 1, NUM_ATOMS, 3]) # [R*8, 3] -> [1, R, 1, 8, 3]
        
        D = np.sqrt(np.sum((A-B)**2, axis=4, keepdims=True)) # [R, R, 8, 8, 1]
        D = np.reshape(D, [Ra,Rb,NUM_ATOMS**2]) # [R, R, 8, 8, 1] -> [R, R, 64]
        
        
    ### assume that input was "mask"
    elif len(array_a_shape)==1:
        Ra = len(array_a)//NUM_ATOMS
        Rb = len(array_b)//NUM_ATOMS
        A = array_a.reshape([Ra, 1, NUM_ATOMS, 1]) # [R*8] -> [R, 1, 8, 1]
        B = array_b.reshape([1, Rb, 1, NUM_ATOMS]) # [R*8] -> [1, R, 1, 8] 
        D = A*B
        D = np.reshape(D, [Ra,Rb,NUM_ATOMS**2])
        
        
    else:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_shape)
    
    
    ### symmetrize
    if Ra==Rb:
        return symmetrize(D)
    return D

def symmetrize(D):
    """
    developed in /alto/chris/projects/main/rna_3d/reconstruct_3d_from_distances/20200728_reverse_symmetrization.ipynb
    input:  D [L,L,64]
    output: D [L,L,64]
    
    If this function is applied to an unsymmetrized tensor, it will be symmetrized.
    If this function is applied to a symmetrized tensor, it will be un-symmetrized!
    
    """    
    upper = np.transpose(np.triu(np.transpose(D,[2,0,1]), 0),[1,2,0])  # 0 includes diagonal
    lower = np.transpose(np.tril(np.transpose(D,[2,0,1]), -1),[1,2,0]) # 0 includes diagonal
    return upper + lower[:,:,symmetrization_indices]


def get_3D_tensor(array_a, *args):
    
    """
    input(s):
        either 
            coords [Ra*8, 3] and optionally [Rb*8, 3]
        or
            mask [Ra*8] and optionally [Rb*8]
        
    output:
        either
            D [Ra, Ra, 64]
        or
            D [Ra, Rb, 64]
    """
    

    if len(args)==1:
        array_b = args[0]
    else:
        array_b = array_a
    
    array_a_shape = array_a.shape
    array_b_shape = array_b.shape
    if array_a_shape[0]%NUM_ATOMS!=0:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_a_shape)
    if array_b_shape[0]%NUM_ATOMS!=0:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_b_shape)        
    
    if len(array_a_shape)!=len(array_b_shape):
        raise Exception('Inputs have different dimensions:', array_a_shape, array_b_shape)  
    
    ### assume that input was "coords"
    if len(array_a_shape)==2:
        Ra = len(array_a)//NUM_ATOMS
        Rb = len(array_b)//NUM_ATOMS
        A = array_a.reshape([Ra, 1, NUM_ATOMS, 1, 3]) # [R*8, 3] -> [R, 1, 8, 1, 3]
        B = array_b.reshape([1, Rb, 1, NUM_ATOMS, 3]) # [R*8, 3] -> [1, R, 1, 8, 3]
        
        D = np.sqrt(np.sum((A-B)**2, axis=4, keepdims=True)) # [R, R, 8, 8, 1]
        D = np.reshape(D, [Ra,Rb,NUM_ATOMS**2]) # [R, R, 8, 8, 1] -> [R, R, 64]
        
        
    ### assume that input was "mask"
    elif len(array_a_shape)==1:
        Ra = len(array_a)//NUM_ATOMS
        Rb = len(array_b)//NUM_ATOMS
        A = array_a.reshape([Ra, 1, NUM_ATOMS, 1]) # [R*8] -> [R, 1, 8, 1]
        B = array_b.reshape([1, Rb, 1, NUM_ATOMS]) # [R*8] -> [1, R, 1, 8] 
        D = A*B
        D = np.reshape(D, [Ra,Rb,NUM_ATOMS**2])
        
        
    else:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_shape)
    
    
    ### symmetrize
    if Ra==Rb:
        triu           = np.reshape(np.triu(np.ones([Ra,Ra]), 1), [Ra,Ra,1])
        triu_plus_diag = np.reshape(np.triu(np.ones([Ra,Ra]), 0), [Ra,Ra,1])
        D              = D*triu_plus_diag + np.transpose(D*triu, axes=[1,0,2])        

    return D

def get_substructure_position(coords, substructure_length):
    
    """
    inputs: 
        (R is the number of residues)
        coords:              [R*8, 3]
        substructure_length: requested number of residues in substructure, must be smaller than R
    
    output:
        tuple consisting of:
            i: integer indicating the starting position of the substructure in the structure
            substructure_length: just the input.
    """
    
    R = len(coords)//NUM_ATOMS
    if R<=substructure_length:    
        return (None, substructure_length)
    else:
        i = np.random.randint(0,R-substructure_length+1) # random index
        return (i, substructure_length)
    
    
    
def get_substructure(coords, mask, i, return_contact_mask=True, cutoff=5):

    """
    Developed 2020-03-30 in:
    /projects/main/rna_3d/data_loaders_and_conversion/20200330_develop_data_augmentation_data_loader.ipynb
    
    inputs:
        (R is the number of residues)
        coords: [R*8, 3]
        mask:   [R*8]
        i:      tuple of (starting position, substructure_length)
        
    outputs:
        (Rs is the number of residues in the substructure)
        coords: [Rs*8, 3]
        mask:   [Rs*8]
        contact_mask: [Rs] (only if return_contact_mask=True)
    """    
    
    (i, Rs) = i
    
    if len(np.shape(coords))!=2 or len(np.shape(mask))!=1:
        raise Exception('Input arrays shoudl have shapes [R*8, 3] and [R*8] but have:', np.shape(coords), np.shape(mask))
    
    lengths = [len(x) for x in [coords, mask]]
    u = np.unique(lengths)
    if len(u)>1:
        raise Exception('Input arrays have different size along dimension 0:', lengths)    
    if u%NUM_ATOMS!=0:
        raise Exception('Input dimension must be divisible by %d but is %d' %(NUM_ATOMS, u))
        
    if i==None:
        return coords, mask
    
    

    else:
        ### randomly select substructure
        sel_substr = np.arange(i*NUM_ATOMS, (i+Rs)*NUM_ATOMS)
        
        if len(sel_substr)%NUM_ATOMS!=0:
            raise Exception(len(sel_substr))

        ### selector for matrix (remaining residues)
        sel_matrix = np.arange(len(coords))
        sel_matrix = np.delete(sel_matrix, sel_substr)

        ### select coords and mask
        coords_substr = coords[sel_substr]
        coords_matrix = coords[sel_matrix]
        mask_substr   = mask[sel_substr]
        mask_matrix   = mask[sel_matrix]

        outputs = [coords_substr, mask_substr]
        
        if return_contact_mask:
            D = get_3D_tensor(coords_substr, coords_matrix) # [R1, R2, 64]
            M = get_3D_tensor(mask_substr,   mask_matrix)   # [R1, R2, 64]

            # Crucial!! This sets masked values to really large numbers!
            # This is because we take the minimum later, but "min" does not know which values to ignore.
            D[M==0] = 10**6 
            D_min   = np.min(D, axis=2) # [R1, R2, 64] -> [R1, R2], use a single atom as a proxy for the whole residue

            contact_mask = np.any(D_min<cutoff, axis=1)*1 # [R1]
            outputs.append(contact_mask)

        return outputs
    
def enc2str(valid_sq):
    vseq = ''.join(enc_seq_pdb.decode(valid_sq[0]))
    s = ''
    for ss in vseq:
        if ss=='<':
            break
        else:
            s+=ss
    return s


def transposeD(tJ):
    tjlist = []
    for m in range(64):
        tmp = (tJ[:,:,m]    + tJ[:,:,m].T) * 0.5
        tjlist.append(tmp)
    return np.stack(tjlist,-1)


def np_pad(matrix, pad_max=100, pad_val=0, max_depth=64):
    M_pad = np.full([pad_max, pad_max, max_depth], pad_val, dtype=float)
    M_pad[:matrix.shape[0], :matrix.shape[1], :] = matrix
    return M_pad


atom_list = ["P","C4'","C2-GA_Pur","C6","N9","C2-CU_Pyr","C4","N1"]
atid = dict(zip(atom_list, np.arange(len(atom_list)))) # atom to index dict
NUM_ATOMS = len(atom_list)
print(NUM_ATOMS)


map_idx_to_atom = dict(zip(np.arange(NUM_ATOMS), ["P","C4'","C2","C6","N9","C2","C4","N1"])) # atom to index dict
atom_types = list(map_idx_to_atom.values())
atom_type_dict = dict(zip(atom_types, [x[0] for x in atom_types]))




def check_check_dict(check_dict):
    """
    Version: 2020-02-17
    Developed in: /projects/main/rna_3d/simrna_drift_dataset/20200212_SimRNA_drift_ramdisk_RMSD_CHECKS.ipynb
    """    
    
    allowed = [set(["P","C4'","C2","C6","N9"]),
               set(["P","C4'","C2","C4","N1"])]
    
    for key, atoms in check_dict.items():
        if not any([set(atoms) == x for x in allowed]):
            print('Residue does not have all necessary atoms', key, atoms)
            return False
    
    return True





def write_8_atom_structure_to_pdb_file(filename, coords, mask, seq, occupancy, temperature_factor):
    """
    Version: 2020-02-17
    Developed in: /projects/main/rna_3d/simrna_drift_dataset/20200212_SimRNA_drift_ramdisk_RMSD_CHECKS.ipynb
    """
    
    if len(coords)//NUM_ATOMS>max_len:
        print('Did not write (structure too > max_len residues)', filename)
        return None
    
    
    lines_list = []
    #lines_list.append('MODEL        1                                                                  \n')

    seq_list = list(seq)
    atom_number = 0
    check_dict = dict()
    
    for k, (c, m, occ, temp) in enumerate(zip(coords, mask, occupancy, temperature_factor)):

        if temp==0:
            temp=10
        
        within_residue_idx  = k % NUM_ATOMS  # 0,1,2,3,4,5,6, 0,1,2,3,4,5,6, 0,1,2,3,4,5,6, ...
        within_sequence_idx = k // NUM_ATOMS # 0,0,0,0,0,0,0, 1,1,1,1,1,1,1, 2,2,2,2,2,2,2, ... 


        ### if no data for first phosphate: append "fake" phosphate
        if k==0 and not m:
            atom_number += 1
            residue_name = seq_list[within_sequence_idx]
            atom_name    = 'P'
            atom_type    = 'P'

            res1_C4prime = coords[1]
            res2_P       = coords[8]
            c            = res1_C4prime + (res1_C4prime-res2_P)

            string_input = (atom_number, atom_name, residue_name, within_sequence_idx+1, 
                            c[0], c[1], c[2], 1.0, 10.0, '', atom_type)
            line = 'ATOM  %5d%5s%4s A%4d    %8.3f%8.3f%8.3f%6.2f%6.2f          %2s%2s\n' % string_input
            lines_list.append(line)  
            
            if within_sequence_idx not in check_dict:
                check_dict[within_sequence_idx] = []
            check_dict[within_sequence_idx].append(atom_name)                

        if m:
            atom_number += 1
            residue_name = seq_list[within_sequence_idx]
            atom_name    = map_idx_to_atom[within_residue_idx]
            atom_type    = atom_type_dict[atom_name]

            string_input = (atom_number, atom_name, residue_name, within_sequence_idx+1, 
                            c[0], c[1], c[2], occ, temp, '', atom_type)
            line = 'ATOM  %5d%5s%4s A%4d    %8.3f%8.3f%8.3f%6.2f%6.2f          %2s%2s\n' % string_input
            lines_list.append(line)

            ### also create last line in case this really is the last line
            string_input = (atom_number+1, '', residue_name, within_sequence_idx+1)                
            last_line = 'TER   %5d%5s%4s A%4d                                                      \n' % string_input
            
            if within_sequence_idx not in check_dict:
                check_dict[within_sequence_idx] = []
            check_dict[within_sequence_idx].append(atom_name)                

    lines_list.append(last_line)
    #lines_list.append('ENDMDL                                                                          \n')
    lines_list.append('END                                                                             \n')
    
    
    if check_check_dict(check_dict):
        if filename is not None:
            with open(filename, 'w') as fileobj:
                for line in lines_list:
                    fileobj.write(line)
        return lines_list

    else:
        print('Did not write', filename)
        return None
    





traindata_df['cluster_size'] = traindata_df['len']
valdata_df['cluster_size']   = valdata_df['len']
testdata_df['cluster_size']  = testdata_df['len']

traindata_df['num_models'] = traindata_df['len']
valdata_df['num_models']   = valdata_df['len']
testdata_df['num_models']  = testdata_df['len']


def conv2d_layer(pad_in, n_filers=None, conv_name=None, k=None, args=None, raw_sq_input=None):
    
    #if k is None:
    #    k = params["resnet_k"]

    pad_val = (k-1)//2
    paddings = tf.constant([[0,0], [pad_val, pad_val], [pad_val, pad_val], [0,0]])
    padded_inputs = tf.pad(tensor=pad_in,
                           paddings=paddings,
                           mode="CONSTANT",
                           constant_values=0)
    
    conv = tf.layers.conv2d(inputs=padded_inputs,
                            filters=n_filers,
                            kernel_size=[k,k],
                            strides=[1,1],
                            padding="VALID",
                            use_bias=False, 
                            kernel_initializer=tf.initializers.variance_scaling(),
                            name=conv_name)
    return conv

def residual_stack(h, num_hiddens, num_residual_layers, num_residual_hiddens):
  h += tf.transpose(h[:,:,:,:],[0,1,2,3])
  h *= 0.5
  for i in range(num_residual_layers):
    h_i = tf.nn.relu(h)

    h_i = conv2d_layer(h_i, num_residual_hiddens, "res3x3_%d" % i, 7)
    
    #snt.Conv2D(
    #    output_channels=num_residual_hiddens,
    #    kernel_shape=(7, 7),
    #    stride=(1, 1),
    #    name="res3x3_%d" % i)(h_i)
    h_i = tf.nn.relu(h_i)
    h_i = conv2d_layer(h_i, num_residual_hiddens, "res1x1_%d" % i, 1)
    #h_i = snt.Conv2D(
    #    output_channels=num_hiddens,
    #    kernel_shape=(1, 1),
    #    stride=(1, 1),
    #    name="res1x1_%d" % i)(h_i)
    h += h_i
  return tf.nn.relu(h)

class Encoder(snt.AbstractModule):
  def __init__(self, num_hiddens, num_residual_layers, num_residual_hiddens,
               name='encoder'):
    super(Encoder, self).__init__(name=name)
    self._num_hiddens = num_hiddens
    self._num_residual_layers = num_residual_layers
    self._num_residual_hiddens = num_residual_hiddens
    
  def _build(self, x):
    h = conv2d_layer(x, self._num_hiddens / 2, "enc_1", 7)
    #h = snt.Conv2D(
    #    output_channels=self._num_hiddens / 2,
    #    kernel_shape=(7, 7),
    #    stride=(1, 1),
    #    name="enc_1")(x)
    h = tf.nn.relu(h)
    h = conv2d_layer(x, self._num_hiddens, "enc_2", 4)
    #h = snt.Conv2D(
    #    output_channels=self._num_hiddens,
    #    kernel_shape=(4, 4),
    #    stride=(1, 1),
    #    name="enc_2")(h)
    h = tf.nn.relu(h) 
    h = conv2d_layer(x, self._num_hiddens, "enc_3", 3)
    #h = snt.Conv2D(
    #    output_channels=self._num_hiddens,
    #    kernel_shape=(3, 3),
    #    stride=(1, 1),
    #    name="enc_3")(h)
    
    h = residual_stack(
        h,
        self._num_hiddens,
        self._num_residual_layers,
        self._num_residual_hiddens)
    return (h)

class Decoder(snt.AbstractModule):
  def __init__(self, num_hiddens, num_residual_layers, num_residual_hiddens,
               name='decoder'):
    super(Decoder, self).__init__(name=name)
    self._num_hiddens = num_hiddens
    self._num_residual_layers = num_residual_layers
    self._num_residual_hiddens = num_residual_hiddens
  
  def _build(self, x):
    h = conv2d_layer(x, self._num_hiddens, "dec_1", 7)
    #h = snt.Conv2D(
    #  output_channels=self._num_hiddens,
    #  kernel_shape=(7, 7),
    #  stride=(1, 1),
    #  name="dec_1")(x)

    h = residual_stack(
        h,
        self._num_hiddens,
        self._num_residual_layers,
        self._num_residual_hiddens)

    h = snt.Conv2DTranspose(
        output_channels=int(self._num_hiddens / 2),
        output_shape=None,
        kernel_shape=(4, 4),
        stride=(1, 1),
        name="dec_2")(h)
    h = tf.nn.relu(h)

    x_recon = snt.Conv2DTranspose(
        output_channels=PARAMS['max_depth'],
        output_shape=None,
        kernel_shape=(2, 2),
        stride=(1, 1),
        name="dec_3")(h)
    #x_recon = conv2d_layer(x_recon, 49, "dec_4", 2)

    return x_recon





# commitment_cost should be set appropriately. It's often useful to try a couple
# of values. It mostly depends on the scale of the reconstruction cost
# (log p(x|z)). So if the reconstruction cost is 100x higher, the
# commitment_cost should also be multiplied with the same amount.
commitment_cost = 0.25
# Use EMA updates for the codebook (instead of the Adam optimizer).
# This typically converges faster, and makes the model less dependent on choice
# of the optimizer. In the VQ-VAE paper EMA updates were not used (but was
# developed afterwards). See Appendix of the paper for more details.
vq_use_ema = True
# This is only used for EMA updates.
decay = 0.99
learning_rate = 3e-5
_BATCH_NORM_DECAY = 0.99
_BATCH_NORM_EPSILON = 1e-3

def batch_norm(inputs, training, data_format, batch_norm_name):
    return tf.layers.batch_normalization(inputs=inputs, momentum=_BATCH_NORM_DECAY, epsilon=_BATCH_NORM_EPSILON, center=True,
                                         scale=True, training=training, fused=True, name=batch_norm_name)

def block_layer(pos, inputs, num_hiddens, training):

    shortcut = inputs
    inputs = batch_norm(inputs, training, 'channels_last', 'batch_norm_' + str(pos) + '_0')
    inputs = tf.nn.relu(inputs)
    inputs = conv2d_layer(inputs, num_hiddens, 'conv_' + str(pos) + '_0', 5)
    inputs = batch_norm(inputs, training, 'channels_last', 'batch_norm_' + str(pos) + '_1')
    inputs = tf.nn.relu(inputs)
    inputs = conv2d_layer(inputs, num_hiddens, 'conv_' + str(pos) + '_1', 3)
    return inputs + shortcut

def dec_block_layer(pos, inputs, num_hiddens):

    shortcut = inputs
    #inputs = batch_norm(inputs, training, 'channels_last', 'batch_norm_' + str(pos) + '_0')
    inputs = tf.nn.relu(inputs)
    inputs = conv2d_layer(inputs, num_hiddens, 'conv_' + str(pos) + '_0', 5)
    #inputs = batch_norm(inputs, training, 'channels_last', 'batch_norm_' + str(pos) + '_1')
    inputs = tf.nn.relu(inputs)
    inputs = conv2d_layer(inputs, num_hiddens, 'conv_' + str(pos) + '_1', 3)
    #resout = inputs + shortcut
    return inputs + shortcut

class Encoder(snt.AbstractModule):
  def __init__(self, num_hiddens, num_residual_layers, num_residual_hiddens,
               name='encoder'):
    super(Encoder, self).__init__(name=name)
    self._num_hiddens = num_hiddens
    self._num_residual_layers = num_residual_layers
    self._num_residual_hiddens = num_residual_hiddens
    
  def _build(self, x, train_ph):
    
    h = conv2d_layer(x, self._num_hiddens*2, "enc_1", 7)
    print(1,h.shape)
    h = tf.nn.relu(h)
    h = conv2d_layer(h, self._num_hiddens, "enc_2", 5)
    print(2,h.shape)

    h = tf.nn.relu(h) 
    h = conv2d_layer(h, self._num_residual_hiddens, "enc_3", 3)
    print(3,h.shape)

    h = block_layer(0, h, self._num_residual_hiddens, train_ph)
    print(4,h.shape)
    #h = block_layer(1, h, self._num_residual_hiddens, train_ph)
    print(5,h.shape)
    #h = block_layer(2, h, self._num_residual_hiddens, train_ph)
    #print(6,h.shape)
    print('----------')
    return h

class Decoder(snt.AbstractModule):
  def __init__(self, num_hiddens, num_residual_layers, num_residual_hiddens,
               name='decoder'):
    super(Decoder, self).__init__(name=name)
    self._num_hiddens = num_hiddens
    self._num_residual_layers = num_residual_layers
    self._num_residual_hiddens = num_residual_hiddens
  
  def _build(self, x, features_mask_placeholder):
    d = conv2d_layer(x, self._num_hiddens / 2, "dec_1", 7)
    print(1,d.shape)
    d = dec_block_layer(3, d, self._num_hiddens / 2)
    print(2,d.shape)
    d = conv2d_layer(d, self._num_hiddens , "dec_2", 5)
    print(3,d.shape)
    #d = tf.nn.relu(d) 
    d = dec_block_layer(4, d, self._num_hiddens )
    print(4,d.shape)
    d = conv2d_layer(d, self._num_hiddens*2 , "dec_3", 5)
    print(5,d.shape)
    d = conv2d_layer(d, self._num_hiddens*4 , "dec_3b", 5)
    print(5,d.shape)
    d = dec_block_layer(5, d, self._num_hiddens*4)
    print(5,d.shape)
    #d = conv2d_layer(d, self._num_residual_hiddens , "dec_4", 3) * features_mask_placeholder
    print(6,d.shape)
    x_recon = dec_block_layer(6, d, self._num_hiddens * 4 ) * features_mask_placeholder
    return tf.nn.relu(x_recon) 

def get_lr(learning_rate, step, lr_decay):
    learning_rate *= lr_decay ** step
    return learning_rate

def get_session(sess):
    session = sess
    while type(session).__name__ != 'Session':
        #pylint: disable=W0212
        session = session._sess
    return session

def np_pad(matrix, pad_max=PARAMS['RNA_len'], pad_val=0, max_depth=64):
    M_pad = np.full([pad_max, pad_max, max_depth], pad_val, dtype=float)
    M_pad[:matrix.shape[0], :matrix.shape[1], :] = matrix
    return M_pad

def np_pad_mask(matrix, pad_max=PARAMS['RNA_len'], pad_val=0, max_depth=64):
    M_pad = np.full([pad_max, pad_max, max_depth], pad_val, dtype=float)
    M_pad[:matrix.shape[0], :matrix.shape[1], :] = matrix
    return M_pad

def batch_data(batch_size):
    idx = [np.random.choice(len(data)) for k in range(batch_size)]
    return np.stack([np_pad(features[_]) for _ in idx]), np.stack([np_pad_mask(features_mask[_]) for _ in idx])
    #return np.stack([np_pad(data['big_matrix_flat'][_]) for _ in idx]), np.stack([np_pad(data['big_mask_flat'][_]) for _ in idx])

def batch_data_seq(batch_size):
    idx = [np.random.choice(len(data)) for k in range(batch_size)]
    return np.stack([seqs[_] for _ in idx]), np.stack([np_pad(features[_]) for _ in idx]), np.stack([np_pad_mask(features_mask[_]) for _ in idx])

def batch_data_seq_nrnd(batch_size, idx2):
    idx = [idx2 for k in range(batch_size)]
    return np.stack([seqs[_] for _ in idx]), np.stack([np_pad(features[_]) for _ in idx]), np.stack([np_pad_mask(features_mask[_]) for _ in idx])

def get_3D_tensor(array_a, *args):
    
    """
    input(s):
        either 
            coords [Ra*8, 3] and optionally [Rb*8, 3]
        or
            mask [Ra*8] and optionally [Rb*8]
        
    output:
        either
            D [Ra, Ra, 64]
        or
            D [Ra, Rb, 64]
    """
    

    if len(args)==1:
        array_b = args[0]
    else:
        array_b = array_a
    
    array_a_shape = array_a.shape
    array_b_shape = array_b.shape
    if array_a_shape[0]%NUM_ATOMS!=0:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_a_shape)
    if array_b_shape[0]%NUM_ATOMS!=0:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_b_shape)        
    
    if len(array_a_shape)!=len(array_b_shape):
        raise Exception('Inputs have different dimensions:', array_a_shape, array_b_shape)  
    
    ### assume that input was "coords"
    if len(array_a_shape)==2:
        Ra = len(array_a)//NUM_ATOMS
        Rb = len(array_b)//NUM_ATOMS
        A = array_a.reshape([Ra, 1, NUM_ATOMS, 1, 3]) # [R*8, 3] -> [R, 1, 8, 1, 3]
        B = array_b.reshape([1, Rb, 1, NUM_ATOMS, 3]) # [R*8, 3] -> [1, R, 1, 8, 3]
        
        D = np.sqrt(np.sum((A-B)**2, axis=4, keepdims=True)) # [R, R, 8, 8, 1]
        D = np.reshape(D, [Ra,Rb,NUM_ATOMS**2]) # [R, R, 8, 8, 1] -> [R, R, 64]
        
        
    ### assume that input was "mask"
    elif len(array_a_shape)==1:
        Ra = len(array_a)//NUM_ATOMS
        Rb = len(array_b)//NUM_ATOMS
        A = array_a.reshape([Ra, 1, NUM_ATOMS, 1]) # [R*8] -> [R, 1, 8, 1]
        B = array_b.reshape([1, Rb, 1, NUM_ATOMS]) # [R*8] -> [1, R, 1, 8] 
        D = A*B
        D = np.reshape(D, [Ra,Rb,NUM_ATOMS**2])
        
        
    else:
        raise Exception('Shape must be [R*8, 3] or [R*8], but is', array_shape)
    
    
    ### symmetrize
    if Ra==Rb:
        triu           = np.reshape(np.triu(np.ones([Ra,Ra]), 1), [Ra,Ra,1])
        triu_plus_diag = np.reshape(np.triu(np.ones([Ra,Ra]), 0), [Ra,Ra,1])
        D              = D*triu_plus_diag + np.transpose(D*triu, axes=[1,0,2])        

    return D

def get_substructure_position(coords, substructure_length):
    
    """
    inputs: 
        (R is the number of residues)
        coords:              [R*8, 3]
        substructure_length: requested number of residues in substructure, must be smaller than R
    
    output:
        tuple consisting of:
            i: integer indicating the starting position of the substructure in the structure
            substructure_length: just the input.
    """
    
    R = len(coords)//NUM_ATOMS
    if R<=substructure_length:    
        return (None, substructure_length)
    else:
        i = np.random.randint(0,R-substructure_length+1) # random index
        return (i, substructure_length)
    
    
    
def get_substructure(coords, mask, i, return_contact_mask=True, cutoff=5):

    """
    Developed 2020-03-30 in:
    /projects/main/rna_3d/data_loaders_and_conversion/20200330_develop_data_augmentation_data_loader.ipynb
    
    inputs:
        (R is the number of residues)
        coords: [R*8, 3]
        mask:   [R*8]
        i:      tuple of (starting position, substructure_length)
        
    outputs:
        (Rs is the number of residues in the substructure)
        coords: [Rs*8, 3]
        mask:   [Rs*8]
        contact_mask: [Rs] (only if return_contact_mask=True)
    """    
    
    (i, Rs) = i
    
    if len(np.shape(coords))!=2 or len(np.shape(mask))!=1:
        raise Exception('Input arrays shoudl have shapes [R*8, 3] and [R*8] but have:', np.shape(coords), np.shape(mask))
    
    lengths = [len(x) for x in [coords, mask]]
    u = np.unique(lengths)
    if len(u)>1:
        raise Exception('Input arrays have different size along dimension 0:', lengths)    
    if u%NUM_ATOMS!=0:
        raise Exception('Input dimension must be divisible by %d but is %d' %(NUM_ATOMS, u))
        
    if i==None:
        return coords, mask
    
    

    else:
        ### randomly select substructure
        sel_substr = np.arange(i*NUM_ATOMS, (i+Rs)*NUM_ATOMS)
        
        if len(sel_substr)%NUM_ATOMS!=0:
            raise Exception(len(sel_substr))

        ### selector for matrix (remaining residues)
        sel_matrix = np.arange(len(coords))
        sel_matrix = np.delete(sel_matrix, sel_substr)

        ### select coords and mask
        coords_substr = coords[sel_substr]
        coords_matrix = coords[sel_matrix]
        mask_substr   = mask[sel_substr]
        mask_matrix   = mask[sel_matrix]

        outputs = [coords_substr, mask_substr]
        
        if return_contact_mask:
            D = get_3D_tensor(coords_substr, coords_matrix) # [R1, R2, 64]
            M = get_3D_tensor(mask_substr,   mask_matrix)   # [R1, R2, 64]

            # Crucial!! This sets masked values to really large numbers!
            # This is because we take the minimum later, but "min" does not know which values to ignore.
            D[M==0] = 10**6 
            D_min   = np.min(D, axis=2) # [R1, R2, 64] -> [R1, R2], use a single atom as a proxy for the whole residue

            contact_mask = np.any(D_min<cutoff, axis=1)*1 # [R1]
            outputs.append(contact_mask)

        return outputs
    
    
def get_drift_path(pdb_id, model_idx, seed=None):
    if seed is None:
        seed = np.random.randint(100)
    return '/sopran/shared/simrna_drift_dataset/20200327_drift_data/%s/model%02d/seed%04d.pkl' % (pdb_id, model_idx, seed)


def load_drift_file(drift_path, wanted_rmsd=1.0):
    if wanted_rmsd not in [1.0, 3.0, 5.0, 10]:
        raise Exception('Invalid drift RMSD requested:', wanted_rmsd)
        
    with open(drift_path, 'rb') as fileobj:
        return pickle.load(fileobj)[wanted_rmsd][0]
    
print('Done')

#PARAMS['batch_size']*PARAMS['N_GPUS']  better use None
g = tf.Graph()
with g.as_default():
    with tf.variable_scope('resnet', reuse=tf.AUTO_REUSE):
        tconfig=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
        with tf.device('/cpu:' + str([x.name for x in device_lib.list_local_devices() if x.device_type == 'CPU'][0][-1])):

            c1 = []
            device_list = [x.name for x in device_lib.list_local_devices() if x.device_type == 'GPU']
            device_list = ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3'] 
            print("device_list: %s \n" % device_list)
            features_placeholder_full      = tf.placeholder(tf.float32, (None ,PARAMS['RNA_len'],PARAMS['RNA_len'] ,PARAMS['max_depth'] ))
            features_mask_placeholder_full = tf.placeholder(tf.float32, (None, PARAMS['RNA_len'],PARAMS['RNA_len'] ,PARAMS['max_depth'] ))
            train_ph                  = tf.placeholder(tf.bool, ())
            seq_ph_full = tf.placeholder(tf.int32, [None])
            step_ph  = tf.placeholder(tf.float32, ())
        start_idx = 0
        #if 1==1:
        all_z  = []
        all_xr = []
        if 1==1:
            #for d in device_list:
            if 1==1:
            #with tf.device(d):
                features_placeholder      = features_placeholder_full[start_idx:start_idx + PARAMS['batch_size']]
                features_mask_placeholder = features_mask_placeholder_full[start_idx:start_idx + PARAMS['batch_size']]
                #train_ph                  = train_ph_full[start_idx:start_idx + PARAMS['batch_size']]
                seq_ph                    = seq_ph_full[start_idx:start_idx + PARAMS['batch_size']]
                #one_mask = tf.ones([seq_ph[0], seq_ph[0]])
                one_list = []
                for m in range(0,PARAMS['batch_size']) :
                    newone_mask = tf.ones([seq_ph[m], seq_ph[m]])
                    paddings = [[0, PARAMS['RNA_len']-tf.shape(newone_mask)[0]], [0, PARAMS['RNA_len']-tf.shape(newone_mask)[0]]]
                    newone_mask = tf.pad(newone_mask, paddings, "CONSTANT")
                    one_list.append(newone_mask)
                    #one_mask = tf.concat([one_mask,newone_mask],axis=1)
                one_mask = tf.stack(one_list)
                one_exp_list = []
                for m in range(0,embedding_dim) :
                    one_exp_list.append(one_mask)
                one_mask_tile = tf.stack(one_exp_list,-1)   
                
                # Build modules.
                #encoder = Encoder(num_hiddens, num_residual_layers, num_residual_hiddens)
                #decoder = Decoder(num_hiddens, num_residual_layers, num_residual_hiddens)

                #pre_vq_conv1 = conv2d_layer(x, embedding_dim, "to_vq", 7)
                encoder = Encoder(num_hiddens, embedding_dim, embedding_dim)
                decoder = Decoder(num_hiddens, num_residual_layers, PARAMS['max_depth'])
                # Process inputs with conv stack, finishing with 1x1 to get to correct size.
                x = features_placeholder * features_mask_placeholder
                #tf.placeholder(tf.float32, shape=(None, image_size, image_size, 3))
                enc_x = encoder(x, train_ph) 

                if vq_use_ema:
                    vq_vae = snt.nets.VectorQuantizerEMA(
                          embedding_dim=embedding_dim,
                          num_embeddings=num_embeddings,
                          commitment_cost=commitment_cost,
                          decay=decay)
                else:
                    vq_vae = snt.nets.VectorQuantizer(
                          embedding_dim=embedding_dim,
                          num_embeddings=num_embeddings,
                          commitment_cost=commitment_cost)

                #zp = pre_vq_conv1(enc_x) 
                #zp = enc_x#conv2d_layer(enc_x, embedding_dim, "to_vq", 3) 
                zp = conv2d_layer(enc_x, embedding_dim, "to_vq", 3) 

                z_T = tf.concat( [tf.stack([tf.reshape(tf.transpose(zp[i,:,:,ilast]),[1,PARAMS['RNA_len'],PARAMS['RNA_len']]) for ilast in range(embedding_dim)], axis=-1) for i in range(batch_size)] , axis=0) 
                z  = 0.5 * (zp + z_T) * one_mask_tile #tf.transpose(zp, [0,1,2,3]))

                # For training
                vq_output_train = vq_vae(z, is_training=True)

                #x_recon_pre = decoder(vq_output_train["quantize"])
                dec_in = vq_output_train["quantize"] * one_mask_tile
                x_recon = decoder(dec_in, features_mask_placeholder) * features_mask_placeholder
                # For evaluation, make sure is_training=False!
                vq_output_eval = vq_vae(z, is_training=False)
                #vq_output_eval_tensor  = vq_output_eval["quantize"]
                #vq_output_eval_tensor += tf.transpose(vq_output_eval_tensor,[0,1,2,3])
                #vq_output_eval_tensor *= 0.5
                xrec = x_recon_eval = decoder(vq_output_eval["quantize"], features_mask_placeholder)
                zenc = tf.reshape(vq_output_eval['encodings'],[PARAMS["batch_size"],100,100,3])
                recon_error = tf.reduce_mean((x_recon - x)**2 * features_mask_placeholder) 
                loss = recon_error + vq_output_train["loss"] 
                c1.append(loss)
                start_idx = start_idx + PARAMS["batch_size"]
                all_z.append( zenc )
                all_xr.append(x_recon_eval)
        zs = tf.concat(all_z,0)
        xr = tf.concat(x_recon_eval,0)
        with tf.device('/cpu:' + str([x.name for x in device_lib.list_local_devices() if x.device_type == 'CPU'][0][-1])):
            tot_loss = tf.add_n(c1) / PARAMS['N_GPUS']
            


            #x_recon_evalp_T = tf.concat( [tf.stack([tf.reshape(tf.transpose(x_recon_evalp[i,:,:,ilast]),[1,PARAMS['RNA_len'],PARAMS['RNA_len']])\
            #                                   for ilast in range(PARAMS['max_depth'])], axis=-1) for i in range(batch_size)] , axis=0) 


            """
            #x_recon_eval =  0.5 * (x_recon_evalp + x_recon_evalp_T)

            # The following is a useful value to track during training.
            # It indicates how many codes are 'active' on average.
            perplexity = vq_output_train["perplexity"] 

            # Create optimizer and TF session.
            #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            #train_op = optimizer.minimize(tot_loss)
            
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            global_step = tf.train.get_global_step()
            tvars = tf.trainable_variables()

            gradients = optimizer.compute_gradients(tot_loss, tvars, colocate_gradients_with_ops=True)
            minimize_op = optimizer.apply_gradients(gradients, global_step=global_step, name="train")
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            train_op = tf.group(minimize_op, update_ops)
            """
            saver = tf.train.Saver( max_to_keep=0)
            init_op = tf.global_variables_initializer()
print('Done setting up graph.')


# # Dataset lodaer

# In[6]:




# In[7]:


# TRAIN

### Longer sequences should be shown more often (b/c of data augmentation)
#when want to sample remaining_ids_sampled ***1
lengths       = np.array(list(traindata_df['len']))
probabilities = lengths #/ np.sum(lengths)

### Sequences in larger clusters should be shown less often
cluster_sizes = np.array(list(traindata_df['cluster_size']))
model_sizes = np.array(list(traindata_df['num_models']))
probabilities = probabilities/cluster_sizes/model_sizes
probabilities = probabilities/np.sum(probabilities)

### Draw sample
pdb_ids = np.array(list(traindata_df.index))
sampled_pdb_ids = np.random.choice(pdb_ids, 50, p=probabilities, replace= False)
#sampled_pdb_ids = list(set(sampled_pdb_ids))[:50]
#print(sampled_pdb_ids)
print(len(sampled_pdb_ids))

# VALIDATION

### Longer sequences should be shown more often (b/c of data augmentation)
#when want to sample remaining_ids_sampled ***1
lengths       = np.array(list(valdata_df['len']))
probabilities_val = lengths #/ np.sum(lengths)

### Sequences in larger clusters should be shown less often
cluster_sizes = np.array(list(valdata_df['cluster_size']))
model_sizes = np.array(list(valdata_df['num_models']))
probabilities_val = probabilities_val/cluster_sizes/model_sizes
probabilities_val = probabilities_val/np.sum(probabilities_val)

### Draw sample
pdb_ids_val = np.array(list(valdata_df.index))
sampled_pdb_ids_val = np.random.choice(pdb_ids_val, 50, p=probabilities_val, replace= False)
#sampled_pdb_ids = list(set(sampled_pdb_ids))[:50]
#print(sampled_pdb_ids)
print(len(sampled_pdb_ids_val))

# TEST

### Longer sequences should be shown more often (b/c of data augmentation)
#when want to sample remaining_ids_sampled ***1
lengths       = np.array(list(testdata_df['len']))
probabilities_test = lengths #/ np.sum(lengths)

### Sequences in larger clusters should be shown less often
cluster_sizes = np.array(list(testdata_df['cluster_size']))
model_sizes = np.array(list(testdata_df['num_models']))
probabilities_test = probabilities_test/cluster_sizes/model_sizes
probabilities_test = probabilities_test/np.sum(probabilities_test)

### Draw sample
pdb_ids_test = np.array(list(testdata_df.index))
#sampled_pdb_ids_test = np.random.choice(pdb_ids_test, 50, p=probabilities_test, replace= False)
#sampled_pdb_ids = list(set(sampled_pdb_ids))[:50]
#print(sampled_pdb_ids)


# In[8]:



def batch_data_train( batch_size = PARAMS['batch_size']*PARAMS['N_GPUS'], trainflag='train'):
    if trainflag == 'train':
        sampled_pdb_ids = np.random.choice(pdb_ids, batch_size, replace= False)
        return traindata_df.loc[sampled_pdb_ids]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in traindata_df.loc[sampled_pdb_ids]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in traindata_df.loc[sampled_pdb_ids]['mask'].values])
    
    if trainflag == 'valid':
        sampled_pdb_ids_val = np.random.choice(pdb_ids_val, batch_size, replace= False)
        return valdata_df.loc[sampled_pdb_ids_val]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in valdata_df.loc[sampled_pdb_ids_val]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in valdata_df.loc[sampled_pdb_ids_val]['mask'].values])       
                                           
    if trainflag == 'test':
        sampled_pdb_ids_test = np.random.choice(pdb_ids_test, batch_size, replace= False)
        return testdata_df.loc[sampled_pdb_ids_test]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in testdata_df.loc[sampled_pdb_ids_test]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in testdata_df.loc[sampled_pdb_ids_test]['mask'].values])        


def iter_data_train( i, batch_size = PARAMS['batch_size']*PARAMS['N_GPUS'], trainflag='train'):
    if trainflag == 'train':
        sampled_pdb_ids = pdb_ids[i*batch_size:(i+1)*batch_size]#np.random.choice(pdb_ids, batch_size, replace= False)
        if len(sampled_pdb_ids)<batch_size:
            sampled_pdb_ids = pdb_ids[-batch_size:]
        return sampled_pdb_ids,traindata_df.loc[sampled_pdb_ids]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in traindata_df.loc[sampled_pdb_ids]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in traindata_df.loc[sampled_pdb_ids]['mask'].values])
    
    if trainflag == 'valid':
        sampled_pdb_ids_val = pdb_ids_val[i*batch_size:(i+1)*batch_size]#np.random.choice(pdb_ids_val, batch_size, replace= False)
        if len(sampled_pdb_ids_val)<batch_size:
            sampled_pdb_ids_val = pdb_ids_val[-batch_size:]
        return sampled_pdb_ids_val,valdata_df.loc[sampled_pdb_ids_val]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in valdata_df.loc[sampled_pdb_ids_val]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in valdata_df.loc[sampled_pdb_ids_val]['mask'].values])       
                                           
    if trainflag == 'test':
        sampled_pdb_ids_test = pdb_ids_test[i*batch_size:(i+1)*batch_size]#np.random.choice(pdb_ids_test, batch_size, replace= False)
        if len(sampled_pdb_ids_test)<batch_size:
            sampled_pdb_ids_test = pdb_ids_test[-batch_size:]
        return sampled_pdb_ids_test,testdata_df.loc[sampled_pdb_ids_test]['len'].values,            
    np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in testdata_df.loc[sampled_pdb_ids_test]['coords'].values]),
    np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in testdata_df.loc[sampled_pdb_ids_test]['mask'].values])        

print('ckpt dir: ', head_ckpt_dir)




5345
8
8
Done
device_list: ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3'] 

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
1 (?, 100, 100, 32)
2 (?, 100, 100, 16)
3 (?, 100, 100, 8)
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
4 (?, 100, 100, 8)
5 (?, 100, 100, 8)
----------
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


1 (?, 100, 100, 8)
2 (?, 100, 100, 8)
3 (?, 100, 100, 16)
4 (?, 100, 100, 16)
5 (?, 100, 100, 32)


5 (?, 100, 100, 64)
5 (?, 100, 100, 64)
6 (?, 100, 100, 64)
1 (?, 100, 100, 8)
2 (?, 100, 100, 8)


3 (?, 100, 100, 16)
4 (?, 100, 100, 16)
5 (?, 100, 100, 32)
5 (?, 100, 100, 64)
5 (?, 100, 100, 64)
6 (?, 100, 100, 64)


Done setting up graph.
50
50
ckpt dir:  /sopran/shared/embeddings/vqvae_3d_100_0707/


In [5]:
#with open( '/gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/L64x64_substructure_23022022.pkl', 'rb') as f:
#    traindata_df, valdata_df, testdata_df = pickle.load( f)
#print(len(traindata_df))

In [6]:
pdb_ids = np.array(list(traindata_df.index))
pdb_ids_val = np.array(list(valdata_df.index))
pdb_ids_test = np.array(list(testdata_df.index))
def pdb_batch_data_train( batch_size = PARAMS['batch_size']*PARAMS['N_GPUS'], trainflag='train'):
    if trainflag == 'train':
        sampled_pdb_ids = np.random.choice(pdb_ids, batch_size, replace= False)
        return sampled_pdb_ids, traindata_df.loc[sampled_pdb_ids]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in traindata_df.loc[sampled_pdb_ids]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in traindata_df.loc[sampled_pdb_ids]['mask'].values])
    
    if trainflag == 'valid':
        sampled_pdb_ids_val = np.random.choice(pdb_ids_val, batch_size, replace= False)
        return sampled_pdb_ids_val,  valdata_df.loc[sampled_pdb_ids_val]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in valdata_df.loc[sampled_pdb_ids_val]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in valdata_df.loc[sampled_pdb_ids_val]['mask'].values])       
                                           
    if trainflag == 'test':
        sampled_pdb_ids_test = np.random.choice(pdb_ids_test, batch_size, replace= False)
        return sampled_pdb_ids_test, testdata_df.loc[sampled_pdb_ids_test]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in testdata_df.loc[sampled_pdb_ids_test]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in testdata_df.loc[sampled_pdb_ids_test]['mask'].values])        

def iter_data_train( i, batch_size = PARAMS['batch_size']*PARAMS['N_GPUS'], trainflag='train'):
    if trainflag == 'train':
        sampled_pdb_ids = pdb_ids[i*batch_size:(i+1)*batch_size]#np.random.choice(pdb_ids, batch_size, replace= False)
        if len(sampled_pdb_ids)<batch_size:
            sampled_pdb_ids = pdb_ids[-batch_size:]
        return sampled_pdb_ids,traindata_df.loc[sampled_pdb_ids]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in traindata_df.loc[sampled_pdb_ids]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in traindata_df.loc[sampled_pdb_ids]['mask'].values])
    
    if trainflag == 'valid':
        sampled_pdb_ids_val = pdb_ids_val[i*batch_size:(i+1)*batch_size]#np.random.choice(pdb_ids_val, batch_size, replace= False)
        if len(sampled_pdb_ids_val)<batch_size:
            sampled_pdb_ids_val = pdb_ids_val[-batch_size:]
        return sampled_pdb_ids_val,valdata_df.loc[sampled_pdb_ids_val]['len'].values,             np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in valdata_df.loc[sampled_pdb_ids_val]['coords'].values]),             np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in valdata_df.loc[sampled_pdb_ids_val]['mask'].values])       
                                           
    if trainflag == 'test':
        sampled_pdb_ids_test = pdb_ids_test[i*batch_size:(i+1)*batch_size]#np.random.choice(pdb_ids_test, batch_size, replace= False)
        if len(sampled_pdb_ids_test)<batch_size:
            sampled_pdb_ids_test = pdb_ids_test[-batch_size:]
        return sampled_pdb_ids_test,testdata_df.loc[sampled_pdb_ids_test]['len'].values,  np.stack([np_pad(get_3D_tensor(tcoord)) for tcoord in testdata_df.loc[sampled_pdb_ids_test]['coords'].values]),np.stack([np_pad_mask(get_3D_tensor(tcoord)) for tcoord in testdata_df.loc[sampled_pdb_ids_test]['mask'].values])        

    
def get_max_dist_neighbour(pdb_weirdo, flag = 'train'):
    if flag == 'train':
        weirdo = get_3D_tensor(traindata_df.loc[pdb_weirdo]['coords'])[:,:,0]
    elif flag == 'valid':
        weirdo = get_3D_tensor(valdata_df.loc[pdb_weirdo]['coords'])[:,:,0]
    elif flag == 'test':
        weirdo = get_3D_tensor(testdata_df.loc[pdb_weirdo]['coords'])[:,:,0]
    check = np.zeros(weirdo.shape) 
    check[10:,10:] += weirdo[10:,10:]
    cut_matrix = weirdo - check
    cut = cut_matrix.max()
    check = np.zeros(weirdo.shape) 
    #check[2:,10:15] += weirdo[2:,10:15]
    check[5:,5:] += weirdo[5:,5:]
    beta = weirdo - check
    beta = beta[4].max()
    
    diag_shift = weirdo.diagonal()[1:] + weirdo[1:,:-1].diagonal()
    
    return cut, beta, diag_shift.max()

# Decode embedding

In [7]:
#!cp /gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/vqvae100_3d_8/ckpt/plots/vqvae_01022022_loss_L100_lre5_emb3_dim8.png .

In [8]:
#!ls -ltr /gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/vqvae100_3d_8/ckpt

In [9]:
"""##### SINGLE MODE #####
head_ckpt_dir = head_sopran_dir = '/gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/vqvae100_3d_8/ckpt/'
z_data = {}

with open('./data/puzzle.pkl', 'rb') as f:
    puzzle, idx_in = pickle.load(f)
    
print(puzzle)
puzzle_seq = np.array([enc_seq_pdb.encode(puzzle) for i in range(PARAMS['batch_size'])])
x_in = np.ones((PARAMS['batch_size'], 100, 100, 64))
mask_in = np.ones((PARAMS['batch_size'], 100, 100, 64))
lseq_in = np.array([len(puzzle) for i in range(PARAMS['batch_size']) ])


    
#head_ckpt_dir = '/gpfs/scratch/ramakers/sopran/shared/embeddings/vqvae_3d_100_0810/tmp/'
#head_sopran_dir = '/gpfs/scratch/ramakers/sopran/vqvae/'
n_iter = 160000
batch_size = 1
plotme = False
full_res = []        
with tf.Session(graph=g, config=tconfig) as sess:
    saver.restore(sess, head_ckpt_dir + "model_L50_15122020_L100_lre5_emb3_dim8_3_train_{0}.ckpt".format(n_iter))
    feed_dict = {vq_output_eval['encoding_indices']: idx_in,features_placeholder_full : x_in,  features_mask_placeholder_full : mask_in , train_ph: False, seq_ph_full:lseq_in }
    results = sess.run(x_recon_eval, feed_dict=feed_dict)

z_data.update({puzzle : [ idx_in[0] , results[0]]})    
plt.imshow(results[0,:,:,0])
plt.show()"""

'##### SINGLE MODE #####\nhead_ckpt_dir = head_sopran_dir = \'/gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/vqvae100_3d_8/ckpt/\'\nz_data = {}\n\nwith open(\'./data/puzzle.pkl\', \'rb\') as f:\n    puzzle, idx_in = pickle.load(f)\n    \nprint(puzzle)\npuzzle_seq = np.array([enc_seq_pdb.encode(puzzle) for i in range(PARAMS[\'batch_size\'])])\nx_in = np.ones((PARAMS[\'batch_size\'], 100, 100, 64))\nmask_in = np.ones((PARAMS[\'batch_size\'], 100, 100, 64))\nlseq_in = np.array([len(puzzle) for i in range(PARAMS[\'batch_size\']) ])\n\n\n    \n#head_ckpt_dir = \'/gpfs/scratch/ramakers/sopran/shared/embeddings/vqvae_3d_100_0810/tmp/\'\n#head_sopran_dir = \'/gpfs/scratch/ramakers/sopran/vqvae/\'\nn_iter = 160000\nbatch_size = 1\nplotme = False\nfull_res = []        \nwith tf.Session(graph=g, config=tconfig) as sess:\n    saver.restore(sess, head_ckpt_dir + "model_L50_15122020_L100_lre5_emb3_dim8_3_train_{0}.ckpt".format(n_iter))\n    feed_dict = {vq_output_eval[\'encoding_indices\

In [10]:
##### MULTI PUZZLE MODE #####
head_ckpt_dir = head_sopran_dir = '/gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/vqvae100_3d_8/ckpt/'
z_data = {}
# att4_zero
#for idx in [1900000, 2000000, 2100000, 2200000, 2300000, 2400000]: noshape_zero
#[1100000, 1200000, 1300000, 1400000,1500000, 1600000] 1975000
idx =    2035000
#with open('./data/att4_zero_{0}.pkl'.format(idx), 'rb') as f:
with open('./data/noshape_zero.pkl'.format(idx), 'rb') as f:
    puzzles = pickle.load(f)
    



    
#head_ckpt_dir = '/gpfs/scratch/ramakers/sopran/shared/embeddings/vqvae_3d_100_0810/tmp/'
#head_sopran_dir = '/gpfs/scratch/ramakers/sopran/vqvae/'
n_iter = 160000
batch_size = 1
plotme = False
full_res = []        
with tf.Session(graph=g, config=tconfig) as sess:
    saver.restore(sess, head_ckpt_dir + "model_L50_15122020_L100_lre5_emb3_dim8_3_train_{0}.ckpt".format(n_iter))
    for pdb in list(puzzles.keys()):
        puzzle, idx_in = puzzles[pdb]
        print(pdb, puzzle)
        puzzle_seq = np.array([enc_seq_pdb.encode(puzzle) for i in range(PARAMS['batch_size'])])
        x_in = np.ones((PARAMS['batch_size'], 100, 100, 64))
        mask_in = np.ones((PARAMS['batch_size'], 100, 100, 64))
        lseq_in = np.array([len(puzzle) for i in range(PARAMS['batch_size']) ])

        feed_dict = {vq_output_eval['encoding_indices']: idx_in,features_placeholder_full : x_in,  features_mask_placeholder_full : mask_in , train_ph: False, seq_ph_full:lseq_in }
        results = sess.run(x_recon_eval, feed_dict=feed_dict)

        z_data.update({pdb : [puzzle, idx_in[0] , results[0]]})    
#plt.imshow(results[0,:,:,0])
#plt.show()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /gpfs/scratch/ramakers/rnafold/data/Lx64x64/31012022/vqvae/vqvae100_3d_8/ckpt/model_L50_15122020_L100_lre5_emb3_dim8_3_train_160000.ckpt
3owz_A GGCUCUGGAGAGAACCGUUUAAUCGGUCGCCGAAGGAGCAAGCUCUGCGGAAACGCAGAGUGAAACUCUCAGGCAAAAGGACAGAGUC
5lyu_B GGGGAUCUGUCACCCCAUUGAUCGCCUUCGGGCUGAUCUGGCUGGCUAGGCGGGUCCC
6jq5_B UUACUGUGAGAAUCAGUAACAAACAUGUGGGGCUUAUAUCUAAUCGAAAGAUUAGUAUUAGUGCAGACGUUAAAACCAUGUC
5di4_A GGGUACUUAAGCCCACUGAUGAGUCGCUGGGAUGCGACGAAACGCCCA
5swd_A GGGAAGAUAUAAUCCUAAUGAUAUGGUUUGGGAGUUUCUACCAAGAGCCUUAAACUCUUGAUUAUCUUCCC
4qlm_A CGCUGAACCCGAAAGGGGCGGGGGACCCAGAAAUGGGGCGAAUCUCUUCCGAAAGGAAGAGUAGGGUUACUCCUUCGACCCGAGCCCGUCAGCUAACCUC
1y26_X CGCUUCAUAUAAUCCUAAUGAUAUGGUUUGGGAGUUUCUACCAAGAGCCUUAAACUCUUGAUUAUGAAGUG
5kpy_A GGACACUGAUGAUCGCGUGGAUAUGGCACGCAUUGAAUUGUUGGACACCGUAAAUGUCCUAACACGUGUCC
6pmo_A GAAAGUGGGUGCGCGUUUGGCGCAUCAACUCGGGUGGAACCGCGGGAGCUACGCUCUCGUCCCGAG
6pom_A 

# Decode and relax into into Lx8 structure

In [11]:
res_n = {}

res = {}

for pdb in list(z_data.keys()):

    #print('Puzzle: ', pdb)
    tseq = testdata_df.loc[pdb]['subseq']
    tlen = len(tseq)

    LxLx64_distance_tensor     = z_data[pdb][-1][:tlen,:tlen]
    LxLx64_distance_truetensor = z_data[pdb][-1][:tlen,:tlen]

    Lx3_coords = testdata_df.loc[pdb]['coords']
    L_mask     = testdata_df.loc[pdb]['mask']

    L8xL8_distance_matrix = LxLx64_tensor_to_L8xL8_matrix(symmetrize_LxLx64(LxLx64_distance_tensor))

    Lx3_coords_reconstructed = L8xL8_matrix_to_Lx3_coords(L8xL8_distance_matrix, L_mask)

    rms,ref_struct, recon_struct = superimposer(Lx3_coords, Lx3_coords_reconstructed, L_mask)
    res.update({pdb:rms})
    
    print()
    print(pdb, tlen, rms)
    print()
    """
    xsize, ysize = 10, 5
    print('preds softmax')
    fig, ax = plt.subplots(1,3, figsize=(xsize,ysize))
    ax[0].imshow(z_data[pdb][1])
    ax[1].imshow(z_data[pdb][-1][:,:,0])
    ax[2].imshow(get_3D_tensor(Lx3_coords)[:,:,0])
    plt.show()

    fig = plt.figure(figsize=[10,10])
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(ref_struct[:,0],ref_struct[:,1],ref_struct[:,2], 'b-o', alpha=0.5)
    ax.plot(recon_struct[:,0],recon_struct[:,1],recon_struct[:,2], 'r:o', alpha=0.5) 
    plt.show()
    """
    res_n.update({n_iter:res.copy()})

    print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
    print()

    #occupancy = np.ones(len(Lx3_coords_reconstructed))
    #temperature_factor = np.zeros(len(Lx3_coords_reconstructed)) 
    #DO_WRITE = write_8_atom_structure_to_pdb_file('./pdbs/'+str(pdb)+'.pdb', Lx3_coords_reconstructed, L_mask, tseq, occupancy, temperature_factor)


3owz_A 88 10.582450699008172

########## 160000 10.582450699008172 0.0


5lyu_B 58 12.36498266602876

########## 160000 11.473716682518466 0.8912659835102943


6jq5_B 82 13.313753217673874

########## 160000 12.087062194236935 1.1322347234851216


5di4_A 48 11.542161446408176

########## 160000 11.950837007279745 1.0085328484901404


5swd_A 71 7.5922839793522465

########## 160000 11.079126401694245 1.9629641679734922


4qlm_A 100 21.5153876081236

########## 160000 12.81850326943247 4.2823104796442255


1y26_X 71 7.984748579006682

########## 160000 12.127966885085929 4.31039274287644


5kpy_A 71 6.927348986797687

########## 160000 11.477889647799898 4.383520594131355


6pmo_A 66 20.280986883009533

########## 160000 12.45601156282319 4.9733293230558635


6pom_A 100 37.73745875295554

########## 160000 14.984156281836425 8.932202728469676


6ufm_B 98 18.426298779513456

########## 160000 15.29707832707979 8.57381679368664


4lck_C 100 16.07244562681452

########## 160000 15.36169226

In [109]:
#1995000 rungen4
for k in sorted(res.keys()):print(k, '\t{:.2f}'.format(res[k]) )
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))

1y26_X 	5.90
3owz_A 	8.10
3owz_B 	12.89
4l81_A 	6.04
4lck_B 	8.55
4lck_C 	14.52
4lck_E 	8.70
4lck_F 	17.96
4p9r_A 	5.17
4qlm_A 	10.46
4r4v_A 	11.41
4xw7_A 	10.33
5di4_A 	7.90
5di4_B 	12.33
5k7c_A 	8.83
5k7c_B 	6.83
5kpy_A 	5.10
5lyu_A 	10.26
5lyu_B 	10.09
5nwq_A 	12.41
5nwq_B 	12.28
5swd_A 	7.10
5swd_B 	6.97
5swe_X 	6.00
5tpy_A 	6.64
5y85_A 	4.62
5y85_B 	9.90
5y85_C 	4.71
5y85_D 	9.92
6jq5_A 	15.83
6jq5_B 	13.50
6ol3_C 	17.35
6p2h_A 	9.74
6pmo_A 	21.02
6pmo_B 	10.55
6pom_A 	26.81
6pom_B 	8.30
6ufm_A 	9.95
6ufm_B 	18.09
########## 160000 10.59128689583909 4.7369726879034895


In [116]:
#1600000 noshape
for k in sorted(res.keys()):print(k, '\t{:.2f}'.format(res[k]) )
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))

1y26_X 	8.55
3owz_A 	10.06
3owz_B 	11.89
4l81_A 	6.49
4lck_B 	12.50
4lck_C 	15.58
4lck_E 	12.51
4lck_F 	21.96
4p9r_A 	4.60
4qlm_A 	21.80
4r4v_A 	8.43
4xw7_A 	9.87
5di4_A 	11.01
5di4_B 	18.61
5k7c_A 	9.29
5k7c_B 	9.14
5kpy_A 	8.26
5lyu_A 	13.45
5lyu_B 	13.21
5nwq_A 	11.01
5nwq_B 	11.03
5swd_A 	8.22
5swd_B 	8.24
5swe_X 	6.89
5tpy_A 	8.96
5y85_A 	8.98
5y85_B 	8.99
5y85_C 	9.19
5y85_D 	9.02
6jq5_A 	15.57
6jq5_B 	13.22
6ol3_C 	15.56
6p2h_A 	9.82
6pmo_A 	18.23
6pmo_B 	12.34
6pom_A 	36.01
6pom_B 	10.81
6ufm_A 	11.47
6ufm_B 	17.29
########## 160000 12.257543907849495 5.49879376009963


In [ ]:
"""
1975000
1y26_X 	5.87
3owz_A 	8.10
3owz_B 	13.02
4l81_A 	6.00
4lck_B 	8.50
4lck_C 	14.70
4lck_E 	8.65
4lck_F 	18.09
4p9r_A 	5.03
4qlm_A 	10.42
4r4v_A 	11.71
4xw7_A 	10.27
5di4_A 	7.77
5di4_B 	12.31
5k7c_A 	8.84
5k7c_B 	6.83
5kpy_A 	5.10
5lyu_A 	10.30
5lyu_B 	10.12
5nwq_A 	12.54
5nwq_B 	12.40
5swd_A 	7.11
5swd_B 	7.21
5swe_X 	5.96
5tpy_A 	6.58
5y85_A 	4.67
5y85_B 	10.02
5y85_C 	4.78
5y85_D 	10.03
6jq5_A 	15.92
6jq5_B 	13.58
6ol3_C 	17.26
6p2h_A 	9.67
6pmo_A 	21.08
6pmo_B 	10.54
6pom_A 	26.92
6pom_B 	8.25
6ufm_A 	10.09
6ufm_B 	18.00
########## 160000 10.621520872363721 4.764119511208484


2400000
1y26_X 	7.90
3owz_A 	9.46
3owz_B 	13.15
4l81_A 	11.72
4lck_B 	23.63
4lck_C 	20.44
4lck_E 	24.04
4lck_F 	23.96
4p9r_A 	6.49
4qlm_A 	17.07
4r4v_A 	13.85
4xw7_A 	12.03
5di4_A 	9.92
5di4_B 	11.21
5k7c_A 	13.53
5k7c_B 	9.31
5kpy_A 	8.84
5lyu_A 	10.15
5lyu_B 	10.05
5nwq_A 	14.97
5nwq_B 	14.84
5swd_A 	10.55
5swd_B 	8.26
5swe_X 	5.83
5tpy_A 	13.29
5y85_A 	7.53
5y85_B 	10.73
5y85_C 	7.59
5y85_D 	10.79
6jq5_A 	16.81
6jq5_B 	15.97
6ol3_C 	20.06
6p2h_A 	13.81
6pmo_A 	26.02
6pmo_B 	13.16
6pom_A 	25.06
6pom_B 	10.88
6ufm_A 	15.70
6ufm_B 	27.87
########## 160000 14.011461400666125 5.795682824852666

2300000
1y26_X 	7.48
3owz_A 	9.70
3owz_B 	12.10
4l81_A 	8.74
4lck_B 	18.13
4lck_C 	18.71
4lck_E 	18.36
4lck_F 	20.75
4p9r_A 	5.79
4qlm_A 	16.26
4r4v_A 	11.71
4xw7_A 	11.97
5di4_A 	9.95
5di4_B 	12.93
5k7c_A 	11.75
5k7c_B 	9.43
5kpy_A 	7.27
5lyu_A 	9.06
5lyu_B 	8.97
5nwq_A 	15.69
5nwq_B 	15.73
5swd_A 	9.36
5swd_B 	7.68
5swe_X 	6.27
5tpy_A 	10.21
5y85_A 	8.40
5y85_B 	10.83
5y85_C 	8.48
5y85_D 	10.81
6jq5_A 	15.38
6jq5_B 	15.38
6ol3_C 	17.66
6p2h_A 	10.46
6pmo_A 	23.45
6pmo_B 	10.85
6pom_A 	26.50
6pom_B 	10.26
6ufm_A 	15.07
6ufm_B 	24.85
########## 160000 12.88154681998209 5.086736964623545

2200000
1y26_X 	6.01
3owz_A 	8.14
3owz_B 	10.25
4l81_A 	6.35
4lck_B 	12.61
4lck_C 	15.43
4lck_E 	12.96
4lck_F 	18.10
4p9r_A 	5.00
4qlm_A 	12.26
4r4v_A 	12.55
4xw7_A 	11.02
5di4_A 	9.76
5di4_B 	13.93
5k7c_A 	11.40
5k7c_B 	9.61
5kpy_A 	5.11
5lyu_A 	9.68
5lyu_B 	9.55
5nwq_A 	12.95
5nwq_B 	12.97
5swd_A 	8.19
5swd_B 	7.02
5swe_X 	5.99
5tpy_A 	7.60
5y85_A 	7.68
5y85_B 	10.60
5y85_C 	7.80
5y85_D 	10.55
6jq5_A 	15.72
6jq5_B 	15.81
6ol3_C 	17.70
6p2h_A 	11.53
6pmo_A 	17.29
6pmo_B 	10.06
6pom_A 	28.46
6pom_B 	8.54
6ufm_A 	12.27
6ufm_B 	19.23
########## 160000 11.479109175466833 4.57017314098255

2100000
1y26_X 	7.47
3owz_A 	7.44
3owz_B 	10.20
4l81_A 	6.93
4lck_B 	11.11
4lck_C 	17.18
4lck_E 	11.26
4lck_F 	20.87
4p9r_A 	4.87
4qlm_A 	14.33
4r4v_A 	11.28
4xw7_A 	10.05
5di4_A 	9.10
5di4_B 	13.37
5k7c_A 	10.36
5k7c_B 	8.86
5kpy_A 	6.15
5lyu_A 	9.54
5lyu_B 	9.35
5nwq_A 	13.67
5nwq_B 	13.69
5swd_A 	8.36
5swd_B 	8.23
5swe_X 	5.43
5tpy_A 	8.07
5y85_A 	4.93
5y85_B 	10.45
5y85_C 	5.06
5y85_D 	10.39
6jq5_A 	14.33
6jq5_B 	14.36
6ol3_C 	18.00
6p2h_A 	10.94
6pmo_A 	18.43
6pmo_B 	10.27
6pom_A 	27.68
6pom_B 	7.91
6ufm_A 	9.54
6ufm_B 	21.06
########## 160000 11.29468299630205 4.874101806539017


2000000
1y26_X 	7.73
3owz_A 	11.06
3owz_B 	11.64
4l81_A 	9.80
4lck_B 	11.56
4lck_C 	20.84
4lck_E 	11.64
4lck_F 	22.76
4p9r_A 	5.85
4qlm_A 	12.83
4r4v_A 	10.47
4xw7_A 	10.48
5di4_A 	10.75
5di4_B 	12.24
5k7c_A 	9.75
5k7c_B 	7.14
5kpy_A 	6.01
5lyu_A 	8.76
5lyu_B 	8.63
5nwq_A 	14.28
5nwq_B 	14.11
5swd_A 	9.84
5swd_B 	9.54
5swe_X 	5.81
5tpy_A 	8.71
5y85_A 	5.37
5y85_B 	12.16
5y85_C 	5.44
5y85_D 	12.18
6jq5_A 	17.67
6jq5_B 	15.69
6ol3_C 	19.65
6p2h_A 	12.49
6pmo_A 	26.11
6pmo_B 	10.24
6pom_A 	25.19
6pom_B 	8.87
6ufm_A 	17.84
6ufm_B 	25.78
########## 160000 12.485229141384623 5.572772716620793

1900000
1y26_X 	6.06
3owz_A 	8.27
3owz_B 	13.14
4l81_A 	6.08
4lck_B 	8.69
4lck_C 	14.71
4lck_E 	8.84
4lck_F 	17.80
4p9r_A 	5.11
4qlm_A 	10.68
4r4v_A 	11.26
4xw7_A 	10.07
5di4_A 	7.80
5di4_B 	12.33
5k7c_A 	8.82
5k7c_B 	6.83
5kpy_A 	5.14
5lyu_A 	10.18
5lyu_B 	9.99
5nwq_A 	12.77
5nwq_B 	12.63
5swd_A 	7.06
5swd_B 	7.08
5swe_X 	5.85
5tpy_A 	6.64
5y85_A 	4.62
5y85_B 	9.87
5y85_C 	4.71
5y85_D 	9.89
6jq5_A 	15.88
6jq5_B 	13.51
6ol3_C 	17.56
6p2h_A 	9.80
6pmo_A 	21.19
6pmo_B 	10.69
6pom_A 	26.67
6pom_B 	8.30
6ufm_A 	10.14
6ufm_B 	18.23
########## 160000 10.63867157782212 4.752127913709705"""

In [10]:
#1955000 rungen4
for k in sorted(res.keys()):print(k, '\t{:.2f}'.format(res[k]) )
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))

1y26_X 	8.42
3owz_A 	11.28
3owz_B 	13.08
4l81_A 	13.69
4lck_B 	23.59
4lck_C 	28.73
4lck_E 	24.01
4lck_F 	29.49
4p9r_A 	8.32
4qlm_A 	17.49
4r4v_A 	14.99
4xw7_A 	10.79
5di4_A 	9.64
5di4_B 	11.56
5k7c_A 	12.84
5k7c_B 	8.68
5kpy_A 	11.44
5lyu_A 	9.90
5lyu_B 	9.79
5nwq_A 	15.06
5nwq_B 	14.94
5swd_A 	11.84
5swd_B 	10.79
5swe_X 	8.05
5tpy_A 	12.79
5y85_A 	7.30
5y85_B 	10.60
5y85_C 	7.36
5y85_D 	10.63
6jq5_A 	20.92
6jq5_B 	18.88
6ol3_C 	25.46
6p2h_A 	13.91
6pmo_A 	23.02
6pmo_B 	15.45
6pom_A 	26.89
6pom_B 	12.42
6ufm_A 	21.86
6ufm_B 	26.21
########## 160000 15.18204196380455 6.43864419113794


In [38]:
#310000 noshape lr4
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 11.375092284751284 4.846355918498879



{'4l81_A': 6.865182801814916,
 '5di4_A': 11.555158477264342,
 '5nwq_A': 14.095523796748436,
 '6ufm_A': 12.530832648494973,
 '5swd_B': 8.154806425077863,
 '6jq5_B': 15.259354607325594,
 '3owz_B': 12.336093541193272,
 '5y85_C': 5.483982784470177,
 '5nwq_B': 14.084759892250593,
 '5k7c_A': 9.536208107035081,
 '6p2h_A': 10.186704017944688,
 '4lck_C': 16.189527808707734,
 '5y85_B': 11.296463141661947,
 '5y85_D': 11.301091744900823,
 '4r4v_A': 10.805838569924191,
 '4p9r_A': 5.405008364494112,
 '5tpy_A': 6.74604618000895,
 '5swe_X': 5.756935358368716,
 '6pmo_A': 20.74025678234635,
 '5lyu_A': 9.782494083586855,
 '3owz_A': 8.404117042364254,
 '6pom_B': 9.64386124456689,
 '5lyu_B': 9.606434590893487,
 '5y85_A': 5.404054978995964,
 '1y26_X': 7.320779688322033,
 '6ufm_B': 20.016082895988642,
 '6pom_A': 27.44579512349206,
 '6ol3_C': 19.92716618600957,
 '5k7c_B': 6.7761105067717216,
 '6pmo_B': 11.735737301838768,
 '4xw7_A': 10.088133362249957,
 '6jq5_A': 15.983817452533502,
 '4lck_F': 17.213369078053

In [35]:
#870000 noshape lr3
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 14.949084288463359 5.157974955091217



{'3owz_B': 17.37867384015171,
 '5lyu_B': 13.230658794551395,
 '5k7c_A': 18.386788801297133,
 '4l81_A': 10.78565950321137,
 '5kpy_A': 8.743476928688606,
 '5di4_A': 22.42769367364306,
 '5y85_B': 15.700335604321424,
 '6pmo_B': 12.739782622410537,
 '6pmo_A': 18.40093622077801,
 '5swd_B': 9.375269011730197,
 '4qlm_A': 14.238766210490436,
 '5nwq_B': 18.417013513103587,
 '6pom_B': 14.065960996804815,
 '6ol3_C': 22.79232144806704,
 '4xw7_A': 15.49146937930254,
 '5y85_A': 8.536171661367534,
 '4r4v_A': 17.248123420215016,
 '6ufm_B': 16.724524121134387,
 '6jq5_B': 13.749213561601644,
 '4lck_B': 13.409416019527546,
 '6ufm_A': 14.666181714800784,
 '5lyu_A': 13.385130056996692,
 '6jq5_A': 20.487975898898203,
 '5k7c_B': 6.721226999101195,
 '5tpy_A': 13.835887637867916,
 '5nwq_A': 18.394130176559408,
 '5swd_A': 8.392015321887493,
 '5y85_D': 15.685969893261118,
 '5di4_B': 15.07246905638972,
 '5y85_C': 8.554174790639554,
 '4lck_E': 13.43314635267177,
 '4lck_F': 23.968457944518462,
 '4lck_C': 20.09354865

In [11]:
#160000 rungen4 masked
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 11.7003980813843 5.296577915963619



{'5di4_A': 12.388673053378138,
 '5y85_D': 10.784572834507708,
 '5y85_A': 5.1726576175091274,
 '5lyu_B': 10.288918135283215,
 '6jq5_B': 16.191754216905803,
 '5nwq_B': 16.55389930270833,
 '6ufm_A': 9.44963797719801,
 '3owz_B': 10.82614553025169,
 '4r4v_A': 11.502206102181619,
 '5lyu_A': 10.23522813663797,
 '5y85_C': 5.264079352131667,
 '5kpy_A': 5.35731060964928,
 '1y26_X': 8.597518126224703,
 '5di4_B': 10.55595890409277,
 '4xw7_A': 10.589228612656393,
 '5tpy_A': 7.751297986343173,
 '5nwq_A': 16.648605562761194,
 '5swe_X': 5.687336145864809,
 '4lck_C': 21.305946838822774,
 '6ufm_B': 19.40537438842397,
 '3owz_A': 10.80414973531385,
 '6pmo_B': 11.192813458798893,
 '4lck_E': 9.487672365979533,
 '4lck_F': 21.649053192873495,
 '5y85_B': 10.799188854167422,
 '4qlm_A': 11.75857872664283,
 '4p9r_A': 4.495704804292752,
 '5swd_A': 8.178779896253756,
 '5k7c_B': 7.007818702314676,
 '4l81_A': 6.113149699768062,
 '6jq5_A': 17.470006579195985,
 '5swd_B': 7.3639591061121825,
 '6ol3_C': 19.14147530942568

In [12]:
#160000 rungen4 masked
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 11.390999627689638 5.16577255607938



{'6p2h_A': 11.269387086115078,
 '5y85_D': 8.991249488664861,
 '6pmo_A': 24.224508982192802,
 '5di4_B': 9.96344767682451,
 '5y85_C': 6.800646510696244,
 '4lck_C': 15.387565609737392,
 '5y85_B': 9.018464438988355,
 '5swd_A': 8.100016124390557,
 '6jq5_A': 17.561522054586153,
 '4r4v_A': 9.355646820856926,
 '6ol3_C': 19.79033065821403,
 '5kpy_A': 5.0354045273729335,
 '4qlm_A': 16.058186976251708,
 '6pom_B': 9.491285959428435,
 '4xw7_A': 10.891065697767049,
 '5tpy_A': 7.468106445046586,
 '6pom_A': 25.653407522024107,
 '5nwq_A': 16.072691934936586,
 '5nwq_B': 16.04409177916679,
 '6ufm_B': 17.475742136566826,
 '5k7c_A': 8.366168133279652,
 '5di4_A': 9.233201260555573,
 '5lyu_A': 9.44454380192437,
 '6pmo_B': 13.522595078861144,
 '4lck_E': 8.850932348142763,
 '5k7c_B': 6.843327263909725,
 '5swe_X': 5.748460184249395,
 '4p9r_A': 4.9995585957601705,
 '5y85_A': 6.755095068625892,
 '3owz_B': 11.396424707942645,
 '3owz_A': 9.008553445959233,
 '4lck_B': 8.759838801852839,
 '5swd_B': 7.633005610864136,

In [15]:
#160000 att50 masked
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 7.666204124821697 3.0492147752603165



{'4lck_F': 8.596702970613386,
 '6ufm_B': 7.055895897355952,
 '5swd_B': 5.969215303100391,
 '4qlm_A': 6.8346957749201485,
 '5swe_X': 4.598959197468485,
 '6pmo_B': 7.820589874689821,
 '6pom_B': 7.804921970110276,
 '6jq5_B': 5.216373698033285,
 '5di4_B': 9.03692764303525,
 '5k7c_B': 7.476494196321595,
 '4p9r_A': 3.9144981445855676,
 '4lck_B': 7.141360074143546,
 '6p2h_A': 6.143843891505795,
 '5lyu_B': 11.161341346514526,
 '1y26_X': 4.920655810401087,
 '5nwq_B': 4.480416202167558,
 '4l81_A': 4.522990017263543,
 '5y85_D': 5.056462668667498,
 '6pmo_A': 15.156656840851285,
 '4lck_C': 8.694441983186438,
 '4lck_E': 9.472210387819361,
 '5tpy_A': 5.182812676066514,
 '5y85_A': 5.985986986264634,
 '5lyu_A': 10.095118190038272,
 '4r4v_A': 8.012835587871946,
 '3owz_A': 12.045915949761664,
 '6jq5_A': 4.288465328401641,
 '5k7c_A': 4.782239224579082,
 '5swd_A': 7.745922111664744,
 '3owz_B': 6.899203738154669,
 '5kpy_A': 5.570819556117491,
 '5nwq_A': 8.102418346556139,
 '6ol3_C': 12.404699198882103,
 '5y

In [12]:
#100000 att50
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 14.367840365059092 5.034719921591547



{'6ufm_A': 12.547738479476871,
 '5tpy_A': 10.818123884125507,
 '6pmo_A': 19.831047467848844,
 '6pom_B': 9.320912459961834,
 '3owz_A': 13.627075033971126,
 '6jq5_B': 20.209630317470452,
 '5y85_A': 11.474242746005203,
 '5kpy_A': 12.473678917689597,
 '4l81_A': 15.272049012327946,
 '4r4v_A': 12.57121251974092,
 '1y26_X': 13.518971576405956,
 '5swd_B': 8.97324260949576,
 '4lck_E': 11.80874052622854,
 '4lck_F': 23.810495983796713,
 '4xw7_A': 13.266554873701619,
 '5lyu_B': 11.788644265793975,
 '5di4_A': 13.040274573536427,
 '5y85_D': 8.253659467190573,
 '6pmo_B': 12.340324737000403,
 '6p2h_A': 14.826786245762664,
 '5k7c_B': 8.61464483186912,
 '5swe_X': 9.415037135554057,
 '4p9r_A': 6.777705182627851,
 '5k7c_A': 11.929539309637292,
 '5y85_B': 8.149829806470327,
 '4lck_B': 11.578102957890884,
 '4qlm_A': 16.184813737139415,
 '6ol3_C': 20.354786542692313,
 '4lck_C': 25.069957976514296,
 '5y85_C': 11.684018426605117,
 '6pom_A': 28.608333906021834,
 '5swd_A': 9.952957540018692,
 '5di4_B': 18.026176

In [26]:
shown_puzzles = ['4r4v', '4p9r', '6jq5', '6pom',# '6ufm',
                 #'6pmo', 
                 '6ol3', 
                 '4qlm' ,'5y85', '5di4', '4xw7', '5tpy', '5k7c', '3owz', 
                 '4l81', #'4lck', 
                 #'5nwq', 
                 '5kpy', '5lyu', '6p2h']
l = []
for k in res.keys():
    
    if k[:4] in shown_puzzles:
        l.append(res[k])
        print(k, res[k])
print('###', np.mean(l), np.std(l))

6p2h_A 11.162280306081845
6pom_A 25.528625577410786
4r4v_A 12.085027174370959
3owz_B 12.153343693272584
5k7c_A 9.60869848967086
3owz_A 10.477141128747956
4xw7_A 8.139719233834818
5y85_B 9.722929716133184
5y85_C 7.7950785290903735
5kpy_A 6.767988562958947
5y85_A 7.802483849009158
6pom_B 9.484222969261541
6ol3_C 20.529720711732146
5lyu_A 8.755519997825482
5lyu_B 8.72732098984014
4p9r_A 6.025912011348834
5k7c_B 6.895466613349384
5y85_D 9.738641620355985
5di4_A 10.29230981082295
5di4_B 10.16451439138719
4qlm_A 16.09903687517746
5tpy_A 7.778533008565817
6jq5_A 18.101763961016193
6jq5_B 18.10235283330317
4l81_A 5.183908075591263
### 11.08490160520636 4.827584534472993


12.413690942271653

In [12]:
#230000 rungen4 with null mask diags
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 11.941556360329182 5.218508662486335



{'4r4v_A': 11.655276232809998,
 '3owz_A': 9.819894327296131,
 '5k7c_A': 9.868117354094762,
 '6jq5_B': 15.8820495610741,
 '4lck_E': 10.635805343537807,
 '4p9r_A': 5.375504909059298,
 '5y85_B': 10.475907219721199,
 '1y26_X': 6.108797704981592,
 '6ufm_A': 12.58050943709695,
 '5kpy_A': 6.330532329547898,
 '4lck_F': 24.94010854693851,
 '5y85_D': 10.428970794038838,
 '5nwq_B': 16.400925440065357,
 '6pom_A': 25.17043780011663,
 '5swe_X': 7.256371968890405,
 '6jq5_A': 16.19862453053055,
 '5y85_A': 6.928278502741838,
 '5swd_B': 9.0429637508126,
 '4lck_C': 17.347391335203426,
 '5lyu_B': 9.26160230804705,
 '4qlm_A': 15.270353978481126,
 '5k7c_B': 6.762871461248681,
 '5swd_A': 8.833381648645629,
 '5di4_B': 12.038752400873793,
 '6pmo_B': 11.699134919578889,
 '4xw7_A': 10.608999868636683,
 '5y85_C': 6.98965734196087,
 '6ol3_C': 20.3963341526994,
 '6ufm_B': 17.32062333222148,
 '5lyu_A': 9.366994008271275,
 '6p2h_A': 10.296270319081879,
 '5di4_A': 9.91612173328536,
 '4lck_B': 10.562884138299959,
 '6po

In [38]:
#120000 rungen4 with null mask diags
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 13.944135413238133 5.047425041814914



{'5nwq_B': 18.799771488716566,
 '5k7c_A': 10.664718040586452,
 '5nwq_A': 18.899523168111305,
 '6jq5_B': 18.972148510715694,
 '5tpy_A': 13.63103290339181,
 '5di4_B': 17.88641391234914,
 '5lyu_A': 11.613208333463161,
 '5y85_A': 6.955586829078792,
 '3owz_A': 11.619343784712678,
 '6pom_A': 26.502237542276976,
 '6pmo_B': 12.529696991169763,
 '4lck_F': 15.093629316244142,
 '5y85_D': 11.929112679772901,
 '4qlm_A': 14.418675155127902,
 '5lyu_B': 11.652424264455979,
 '6ufm_A': 16.54403464453679,
 '5swd_A': 7.190932347075438,
 '6pom_B': 10.918828537276982,
 '6ufm_B': 24.160862288993215,
 '5swe_X': 9.61324792201622,
 '4l81_A': 7.73814656902417,
 '4xw7_A': 17.215764169431846,
 '6pmo_A': 17.24098486226244,
 '4r4v_A': 14.037952406877384,
 '5y85_C': 7.113056785731472,
 '6p2h_A': 13.504553189600395,
 '1y26_X': 8.531686268758994,
 '4lck_B': 19.463043055894556,
 '5k7c_B': 8.280843938868959,
 '4lck_C': 17.9206263545709,
 '5kpy_A': 11.317361674107017,
 '5swd_B': 8.789793239917287,
 '4p9r_A': 5.48189267593

In [34]:
#120000 rungen4
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 13.93559509296611 5.082216676373168



{'6p2h_A': 13.47564888785534,
 '6ol3_C': 23.485001179384454,
 '5kpy_A': 11.348170329143967,
 '4lck_C': 17.96637881651529,
 '6pom_B': 10.915038739882592,
 '6jq5_B': 19.049956119609842,
 '5y85_D': 11.914167728699523,
 '1y26_X': 8.598647860470129,
 '6jq5_A': 18.428626368819227,
 '5swd_A': 7.090107791434572,
 '5lyu_A': 11.619618790817524,
 '4l81_A': 7.712101658388376,
 '6ufm_B': 24.123579660947787,
 '3owz_B': 13.518474059843795,
 '5swd_B': 8.751038175480419,
 '6pmo_A': 17.280945479192415,
 '5di4_B': 18.06733380415101,
 '4xw7_A': 17.188996712297644,
 '5k7c_A': 10.549826790569977,
 '5nwq_A': 18.906933287677237,
 '5lyu_B': 11.655286271974532,
 '4r4v_A': 14.048222112654102,
 '5y85_A': 7.004203711769056,
 '6pom_A': 26.54792275352379,
 '4p9r_A': 5.427910086825125,
 '4qlm_A': 14.509821342021374,
 '4lck_E': 19.76631446704196,
 '4lck_F': 15.097374423962655,
 '5k7c_B': 7.718946369988662,
 '5tpy_A': 13.61519112182288,
 '3owz_A': 11.686286560875345,
 '6ufm_A': 16.552428805083167,
 '4lck_B': 19.4447510

In [89]:
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 12.78105336949454 4.207511695648259



{'6pmo_A': 17.118501844586373,
 '5y85_C': 9.621299418869478,
 '4qlm_A': 14.046460788830224,
 '5nwq_A': 16.757827237942955,
 '5tpy_A': 12.336262454522418,
 '4r4v_A': 10.205552032180917,
 '5y85_D': 10.939521502360687,
 '5swd_B': 9.331508537156392,
 '4p9r_A': 5.602169136142471,
 '5swe_X': 8.850249695006783,
 '5k7c_A': 11.020122172088612,
 '6p2h_A': 12.922492258391468,
 '1y26_X': 8.492427958483606,
 '6ufm_B': 23.09119212316719,
 '4lck_B': 14.235353079669336,
 '3owz_A': 10.583184894178139,
 '5di4_A': 12.278357598474983,
 '5nwq_B': 16.588587230009324,
 '5lyu_A': 11.496128761632443,
 '4lck_F': 21.201378872047815,
 '6pom_B': 10.108319578665682,
 '6ufm_A': 12.981423177567223,
 '4xw7_A': 15.436635088193722,
 '5y85_B': 10.969312002359196,
 '5di4_B': 18.962410600213413,
 '6ol3_C': 19.68255435331192,
 '5swd_A': 8.392233774977129,
 '5kpy_A': 8.887638295582791,
 '4l81_A': 8.511443248729003}

In [86]:
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()
res

########## 160000 12.722257769400764 4.200617323752026



{'5y85_A': 8.162658680374559,
 '5tpy_A': 13.581430060246632,
 '6jq5_B': 17.91444273329207,
 '4lck_C': 16.718266159018686,
 '5swe_X': 6.399720300877206,
 '6p2h_A': 11.06507848366936,
 '5k7c_A': 12.215076007905743,
 '5y85_B': 9.662105426251722,
 '5lyu_A': 11.435556177968795,
 '6pmo_A': 18.108203922747187,
 '4lck_F': 17.400794700866882,
 '5nwq_A': 17.504133576885767,
 '5y85_C': 8.260480619651252,
 '6ufm_B': 20.14115673597179,
 '3owz_A': 10.464380436155002,
 '4lck_B': 14.916529020860144,
 '5k7c_B': 8.6056062420164,
 '5swd_B': 7.721676150860696,
 '6ufm_A': 16.414178127192066,
 '4lck_E': 15.174924043988444,
 '4r4v_A': 10.779501602799002,
 '5di4_A': 10.715068111262156,
 '5di4_B': 17.75754821988972,
 '6jq5_A': 18.279825160330343,
 '5swd_A': 7.442270909105919,
 '4l81_A': 9.017466186620466,
 '4p9r_A': 5.8905866048683135,
 '4qlm_A': 14.474553141545046}

In [81]:
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))
print()


########## 160000 6.779386511209554 1.3530090918963067



In [82]:
res

{'3owz_B': 8.174813433772364,
 '4lck_E': 7.258030091716915,
 '5y85_C': 6.959630318775557,
 '6jq5_A': 5.133619477562675,
 '5di4_B': 8.36584037054996,
 '5nwq_B': 4.60578503303183,
 '5kpy_A': 5.5416394850232695,
 '5y85_B': 5.877470503824932,
 '4r4v_A': 9.556602352827788,
 '3owz_A': 6.559220041639288,
 '4lck_C': 6.836326949835963,
 '4xw7_A': 6.976583195176355,
 '4lck_F': 9.139404773149908,
 '5nwq_A': 8.738278162704315,
 '6jq5_B': 5.486782702092725,
 '5di4_A': 4.447543348362462,
 '6p2h_A': 5.67788149452197,
 '5tpy_A': 6.095741550758288,
 '4lck_B': 6.967497814568614,
 '6pmo_A': 6.769170134519883,
 '4qlm_A': 6.911077887114852,
 '5k7c_A': 6.460753758108421,
 '4p9r_A': 4.81422366871124,
 '6ufm_A': 7.8092851141896755,
 '6pmo_B': 8.690013800378845,
 '5swd_A': 6.848194642907476,
 '5swd_B': 6.342025696832388}

In [78]:
print('##########', n_iter, np.mean(list(res.values())), np.std(list(res.values())))

########## 160000 7.529652119241821 2.614057022786943


In [77]:
res

{'4lck_F': 10.552308250701083,
 '6pom_B': 6.660278124326192,
 '5di4_B': 7.84693246065069,
 '5tpy_A': 5.751722433153528,
 '6jq5_A': 7.744868005288602,
 '6p2h_A': 5.894041788180107,
 '5di4_A': 4.369317059208708,
 '4r4v_A': 7.662352036233554,
 '6pmo_B': 8.850689995466935,
 '4p9r_A': 4.190104069620308,
 '6jq5_B': 5.132604388039205,
 '6pom_A': 13.146860709025939,
 '5lyu_A': 9.134855547994437,
 '5y85_D': 10.075328722730633,
 '3owz_B': 6.20346812374688,
 '5k7c_A': 5.504099754435188,
 '5swd_A': 6.666351762942676,
 '6ol3_C': 11.422838018643647,
 '5swe_X': 4.885611033239375,
 '6ufm_B': 4.924021702845575,
 '5lyu_B': 10.269051040766845,
 '4xw7_A': 7.8589510405454375,
 '6ufm_A': 5.7646805842937905,
 '5kpy_A': 5.128534022348128,
 '3owz_A': 4.929328764030417,
 '1y26_X': 4.281492799334667,
 '4lck_C': 12.413704737075747,
 '6pmo_A': 11.444848017756922,
 '5y85_B': 9.650666465387575}